In [1]:
!pip install boto3

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install load_dotenv

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import boto3
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings("ignore")

In [4]:
load_dotenv('env.s3creds')

True

s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [5]:
s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [6]:
# Verify that the variables are correctly retrieved (optional)
print(f"Client Access: {s3}")


Client Access: s3.ServiceResource()


In [7]:
# List all buckets
for bucket in s3.buckets.all():
    print(bucket.name)

prj4


In [8]:
for obj in s3.Bucket('prj4').objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='prj4', key='StormEvents_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='eaglei_outages_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='stormOutages_df.csv')


In [9]:
obj = s3.Bucket('prj4').Object('stormOutages_df.csv').get()
file = pd.read_csv(obj['Body'], index_col=0)

df = pd.DataFrame(file)

In [10]:
df.head(5)


,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out
END_YEARMONTH,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52


In [11]:
df = df.assign(fips_code=df.index)

In [12]:
df.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
END_YEARMONTH,,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [13]:
df = df.reset_index(drop=True)

In [14]:
df.shape

(1007790, 8)

In [15]:
customer_counts = df['customers_out'].value_counts()

# Print the result
print(customer_counts)

1        261368
2         91848
3         48512
4         35958
5         32465
          ...  
10674         1
4032          1
4010          1
4004          1
4196          1
Name: customers_out, Length: 7944, dtype: int64


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007790 entries, 0 to 1007789
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   EVENT_ID       1007790 non-null  int64 
 1   STATE          1007790 non-null  object
 2   county         1007790 non-null  object
 3   YEAR           1007790 non-null  int64 
 4   EVENT_TYPE     1007790 non-null  object
 5   FIPS           1007790 non-null  int64 
 6   customers_out  1007790 non-null  int64 
 7   fips_code      1007790 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 61.5+ MB


In [17]:
df.columns

Index(['EVENT_ID', 'STATE', 'county', 'YEAR', 'EVENT_TYPE', 'FIPS',
       'customers_out', 'fips_code'],
      dtype='object')

In [18]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

### Step 3: Split the data into training and testing datasets by using `train_test_split`.

Due to the dataset containing many rows of value "1" for the amount of customers out, that value cannot be used to train in the data set. We will clean the data to remove the counts of "1" for customers out in the data. 

Create the labels set (`y`)  from the customers out value column, and then create the features (`X`) DataFrame from the remaining columns.

In [19]:
df_filtered = df.query("customers_out > 10")
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [20]:
df_filtered['fips_code'] = df_filtered['fips_code'].astype(str)
df_filtered['month'] = df_filtered['fips_code'].str[-2:]
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11


### Step 2: Create the labels set (`y`)  from the “customers_out” column, and then create the features (`X`) DataFrame from the remaining columns.

### we need to convert string/char data types to a coded numerical value in order to translate the data into a usable format for logistical regression. 
## we will take all pertinent data such in str format such as "STATE" and "county" then concatenate them so they are unique.  -- NVM We can use FIPS !! 
# ## once they are merged, we can assign it a numerical value in alphabetical order to render in the model as a "Coded" location based value. --AGAIN  -- NVM We can use FIPS !! 
## EVENT_ID is garbage probably, DELETE 
### EVENT_TYPE Should be classified by intensity of the storm type in ASC order. yay :) 

In [21]:
event_type_counts = df['EVENT_TYPE'].value_counts()
event_type_counts

Winter Weather             205698
Thunderstorm Wind          133259
Winter Storm               106094
Flood                       88426
Strong Wind                 76272
Heavy Snow                  60019
Heavy Rain                  52016
Flash Flood                 50925
Dense Fog                   41227
High Wind                   38363
Drought                     33961
Hail                        26071
Lake-Effect Snow            24936
Tornado                     17513
Cold/Wind Chill              9538
Frost/Freeze                 8476
Funnel Cloud                 8263
Lightning                    7851
Coastal Flood                7829
Debris Flow                  3393
Seiche                       2997
Ice Storm                    2806
Wildfire                     1518
Extreme Cold/Wind Chill       279
Avalanche                      50
Blizzard                        8
Sleet                           2
Name: EVENT_TYPE, dtype: int64

In [22]:
# Define a dictionary to map weather events to scores
event_scores = {
    "Winter Weather": 2,  # Moderate impact
    "Thunderstorm Wind": 5,  # Can cause damage
    "Winter Storm": 7,  # Significant disruptions
    "Flood": 8,  # Major infrastructure damage
    "Strong Wind": 4,  # Potential for damage
    "Heavy Snow": 6,  # Disruptions to travel and infrastructure
    "Heavy Rain": 5,  # Can cause flooding and disruptions
    "Flash Flood": 8,  # Life-threatening and destructive
    "Dense Fog": 3,  # Can cause travel delays and accidents
    "High Wind": 4,  # Potential for damage
    "Drought": 7,  # Long-term impact on agriculture and water resources
    "Hail": 5,  # Can damage property and crops
    "Lake-Effect Snow": 6,  # Can cause heavy snowfall in localized areas
    "Tornado": 9,  # Devastating impact on property and lives
    "Cold/Wind Chill": 3,  # Discomfort and health risks for vulnerable populations
    "Frost/Freeze": 2,  # Can damage crops and vegetation
    "Funnel Cloud": 1,  # Not always associated with a tornado
    "Lightning": 3,  # Can cause fires and injuries
    "Coastal Flood": 8,  # Damage to coastal infrastructure and property
    "Debris Flow": 8,  # Rapid movement of debris can be destructive
    "Seiche": 2,  # Water level fluctuations, typically less impactful
    "Ice Storm": 7,  # Power outages and damage to infrastructure
    "Wildfire": 9,  # Widespread destruction of property and ecological damage
    "Extreme Cold/Wind Chill": 8,  # Life-threatening conditions
    "Avalanche": 9,  # Destructive and can cause fatalities
    "Blizzard": 8,  # Severe travel disruptions and potential infrastructure damage
    "Sleet": 3  # Mix of rain and snow, can cause slippery conditions
}

# Assign scores to the "EVENT_TYPE" column and create a new "domain" column
df_filtered["storm_intensity"] = df_filtered["EVENT_TYPE"].apply(lambda event: event_scores.get(event, 1))  # Assign 1 for missing values

# Print the modified DataFrame (optional)
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month,storm_intensity
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5


In [ ]:

# Adding some identifier values for the model to interpret "outage events" with a general threshold of 500 customers or more to indicate an outage event.
df_filtered['outage_event'] = df_filtered['customers_out'].apply(lambda x: 0 if x <= 500 else 1)

print(df_filtered)


In [23]:
sample_df = df_filtered.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed

In [24]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=sample_df['customers_out']

# Separate the X variables from the features
x = sample_df.drop(columns=['customers_out', 'EVENT_ID', 'fips_code', 'STATE', 'county', 'EVENT_TYPE'])

In [25]:
y.head()

0    50
1    50
2    52
3    52
4    52
Name: customers_out, dtype: int64

In [26]:
y.value_counts()

11      21800
12      12261
13      10230
14       8569
15       8069
        ...  
3631        1
2920        1
2921        1
1727        1
4196        1
Name: customers_out, Length: 7934, dtype: int64

In [27]:
x.head()

,YEAR,FIPS,month,storm_intensity
0,2014,45079201411,11,5
1,2014,45079201411,11,5
2,2014,45079201411,11,5
3,2014,45079201411,11,5
4,2014,45079201411,11,5


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [28]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
### We had to remove the stratify = y argument due to multiple values in our testing data of having only one occurrence. The testing/training of this method
### requires multiple occurrences of the customer out value to train against, and one example will not allow the tool to analyze and understand the difference and 
## potential values to expect given the other values/parameters of our "X" data 
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)
X_train.shape

(338284, 4)

In [29]:
y_train.shape

(338284,)

In [30]:
y_test.shape

(112762,)

## Create a Logistic Regression Model with the Original Data
#####  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [32]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
Logistic_regr = LogisticRegression(solver='lbfgs',max_iter=10,random_state=1)
Logistic_regr

# Fit the model using training data
Logistic_regr.fit(X_train,y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
prediction = Logistic_regr.predict(X_test)
prediction_df = pd.DataFrame({"Prediction": prediction, "Actual": y_test}).reset_index(drop=True)
prediction_df.head()

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd

In [ ]:
# Generate a confusion matrix for the model
conf_mat = confusion_matrix(y_test, prediction)
#conf_mat_df = pd.DataFrame(conf_mat, index= [["Base 0 ", "Base 1 "], columns=["Predicted 0", "Predicted 1"]])
accuracy_score = accuracy_score(y_test, prediction)

NameError: name 'prediction' is not defined

In [ ]:
from sklearn.metrics import classification_report

# Assuming model is your trained model and X_test is your test dataset
y_true = y_test  # True labels
y_pred = Logistic_regr.predict(X_test)  # Predicted labels

# Ensure y_pred is an array
y_pred = y_pred.tolist() if not isinstance(y_pred, (list, np.ndarray)) else y_pred

# Print the classification report for the model
# Displaying results
print("Confusion Matrix Chart")
display(conf_mat)
print(f"Accuracy Score : {accuracy_score}")
print("Classifications")
report = classification_report(y_true, y_pred)
print(report)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

Confusion Matrix Chart
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Accuracy Score : 0.006614607257694074
Classifications
              precision    recall  f1-score   support

        1001       0.00      0.00      0.00         1
        1002       0.00      0.00      0.00         6
        1003       0.00      0.00      0.00        10
        1004       0.00      0.00      0.00         6
        1005       0.00      0.00      0.00        12
        1006       0.00      0.00      0.00         9
        1007       0.00      0.00      0.00         2
        1008       0.00      0.00      0.00         6
        1009       0.00      0.00      0.00         4
        1010       0.00      0.00      0.00        15
        1011       0.00      0.00      0.00        16
        1012       0.00      0.00      0.00         7
        1013       0.00      0.00      0.00         6
        1014       0.00      0.00      0.00        10
        1015       0.00      0.00      0.00        10
        1016       0.00      0.00      0.00        11
        1017       0.00      0.00      0.00        12
        1018       0.00      0.00      0.00         4
        1019       0.00      0.00      0.00        15
        1020       0.00      0.00      0.00         7
        1021       0.00      0.00      0.00         4
        1022       0.00      0.00      0.00        14
        1023       0.00      0.00      0.00         6
        1024       0.00      0.00      0.00         7
        1025       0.00      0.00      0.00         7
        1026       0.00      0.00      0.00        30
        1027       0.00      0.00      0.00         4
        1028       0.00      0.00      0.00         9
        1029       0.00      0.00      0.00        10
        1030       0.00      0.00      0.00         5
        1031       0.00      0.00      0.00        15
        1032       0.00      0.00      0.00         3
        1033       0.00      0.00      0.00         7
        1034       0.00      0.00      0.00        24
        1035       0.00      0.00      0.00        10
        1036       0.00      0.00      0.00         2
        1037       0.00      0.00      0.00         9
        1038       0.00      0.00      0.00         8
        1039       0.00      0.00      0.00         7
        1040       0.00      0.00      0.00         3
        1041       0.00      0.00      0.00         6
        1042       0.00      0.00      0.00         5
        1043       0.00      0.00      0.00        10
        1044       0.00      0.00      0.00         5
        1045       0.00      0.00      0.00         8
        1046       0.00      0.00      0.00         6
        1047       0.00      0.00      0.00         7
        1048       0.00      0.00      0.00         7
        1049       0.00      0.00      0.00         5
        1050       0.00      0.00      0.00         9
        1051       0.00      0.00      0.00        10
        1052       0.00      0.00      0.00         7
        1053       0.00      0.00      0.00         8
        1054       0.00      0.00      0.00         9
        1055       0.00      0.00      0.00        37
        1056       0.00      0.00      0.00        15
        1057       0.00      0.00      0.00         5
        1058       0.00      0.00      0.00         9
        1059       0.00      0.00      0.00         2
        1060       0.00      0.00      0.00        17
        1061       0.00      0.00      0.00         5
        1062       0.00      0.00      0.00        14
        1063       0.00      0.00      0.00         6
        1064       0.00      0.00      0.00        27
        1065       0.00      0.00      0.00         6
        1066       0.00      0.00      0.00        14
        1067       0.00      0.00      0.00         6
        1069       0.00      0.00      0.00         9
        1070       0.00      0.00      0.00         6
        1071       0.00      0.00      0.00         3
        1072       0.00      0.00      0.00         3
        1073       0.00      0.00      0.00        12
        1074       0.00      0.00      0.00         8
        1075       0.00      0.00      0.00         8
        1076       0.00      0.00      0.00         6
        1077       0.00      0.00      0.00         7
        1078       0.00      0.00      0.00         3
        1079       0.00      0.00      0.00         9
        1080       0.00      0.00      0.00         4
        1081       0.00      0.00      0.00         1
        1082       0.00      0.00      0.00        11
        1083       0.00      0.00      0.00         4
        1084       0.00      0.00      0.00         6
        1085       0.00      0.00      0.00         1
        1086       0.00      0.00      0.00         6
        1087       0.00      0.00      0.00        10
        1088       0.00      0.00      0.00         8
        1089       0.00      0.00      0.00         1
        1090       0.00      0.00      0.00        21
        1091       0.00      0.00      0.00        24
        1092       0.00      0.00      0.00         7
        1093       0.00      0.00      0.00         2
        1094       0.00      0.00      0.00         7
        1095       0.00      0.00      0.00         4
        1096       0.00      0.00      0.00         8
        1097       0.00      0.00      0.00        13
        1098       0.00      0.00      0.00         8
        1099       0.00      0.00      0.00         4
        1100       0.00      0.00      0.00         2
        1101       0.00      0.00      0.00         4
        1102       0.00      0.00      0.00         2
        1103       0.00      0.00      0.00         3
        1104       0.00      0.00      0.00         4
        1106       0.00      0.00      0.00         1
        1107       0.00      0.00      0.00        12
        1108       0.00      0.00      0.00         2
        1109       0.00      0.00      0.00         8
        1110       0.00      0.00      0.00         4
        1111       0.00      0.00      0.00         4
        1112       0.00      0.00      0.00         8
        1113       0.00      0.00      0.00         8
        1114       0.00      0.00      0.00        12
        1115       0.00      0.00      0.00         4
        1116       0.00      0.00      0.00         8
        1117       0.00      0.00      0.00        18
        1119       0.00      0.00      0.00         5
        1120       0.00      0.00      0.00         2
        1121       0.00      0.00      0.00        13
        1122       0.00      0.00      0.00         7
        1123       0.00      0.00      0.00         4
        1124       0.00      0.00      0.00         7
        1125       0.00      0.00      0.00         6
        1126       0.00      0.00      0.00         7
        1127       0.00      0.00      0.00         7
        1128       0.00      0.00      0.00        15
        1129       0.00      0.00      0.00         7
        1130       0.00      0.00      0.00         5
        1131       0.00      0.00      0.00         5
        1132       0.00      0.00      0.00         9
        1133       0.00      0.00      0.00        14
        1134       0.00      0.00      0.00         4
        1135       0.00      0.00      0.00         2
        1136       0.00      0.00      0.00         3
        1137       0.00      0.00      0.00         2
        1138       0.00      0.00      0.00         9
        1139       0.00      0.00      0.00        15
        1140       0.00      0.00      0.00        31
        1141       0.00      0.00      0.00        10
        1142       0.00      0.00      0.00         8
        1143       0.00      0.00      0.00         8
        1144       0.00      0.00      0.00         2
        1145       0.00      0.00      0.00         2
        1146       0.00      0.00      0.00         2
        1147       0.00      0.00      0.00        17
        1148       0.00      0.00      0.00         4
        1149       0.00      0.00      0.00         6
        1150       0.00      0.00      0.00         5
        1151       0.00      0.00      0.00        11
        1152       0.00      0.00      0.00         2
        1153       0.00      0.00      0.00         2
        1154       0.00      0.00      0.00         7
        1155       0.00      0.00      0.00         5
        1156       0.00      0.00      0.00         2
        1157       0.00      0.00      0.00         2
        1158       0.00      0.00      0.00         2
        1159       0.00      0.00      0.00        12
        1160       0.00      0.00      0.00         7
        1161       0.00      0.00      0.00         8
        1162       0.00      0.00      0.00        28
        1163       0.00      0.00      0.00        10
        1164       0.00      0.00      0.00        19
        1165       0.00      0.00      0.00        12
        1166       0.00      0.00      0.00         1
        1167       0.00      0.00      0.00         5
        1168       0.00      0.00      0.00        16
        1169       0.00      0.00      0.00         4
        1170       0.00      0.00      0.00         2
        1171       0.00      0.00      0.00         5
        1172       0.00      0.00      0.00         5
        1173       0.00      0.00      0.00         8
        1174       0.00      0.00      0.00         1
        1175       0.00      0.00      0.00         6
        1176       0.00      0.00      0.00         5
        1177       0.00      0.00      0.00         1
        1178       0.00      0.00      0.00        10
        1179       0.00      0.00      0.00         8
        1180       0.00      0.00      0.00         6
        1181       0.00      0.00      0.00         5
        1182       0.00      0.00      0.00        20
        1183       0.00      0.00      0.00         9
        1184       0.00      0.00      0.00        13
        1185       0.00      0.00      0.00        23
        1186       0.00      0.00      0.00        13
        1187       0.00      0.00      0.00        23
        1188       0.00      0.00      0.00         3
        1189       0.00      0.00      0.00         3
        1190       0.00      0.00      0.00         5
        1191       0.00      0.00      0.00         5
        1192       0.00      0.00      0.00         4
        1193       0.00      0.00      0.00         4
        1194       0.00      0.00      0.00         1
        1195       0.00      0.00      0.00         8
        1196       0.00      0.00      0.00        11
        1197       0.00      0.00      0.00        12
        1198       0.00      0.00      0.00         3
        1199       0.00      0.00      0.00         8
        1200       0.00      0.00      0.00         9
        1201       0.00      0.00      0.00         5
        1202       0.00      0.00      0.00         9
        1203       0.00      0.00      0.00         1
        1204       0.00      0.00      0.00        20
        1205       0.00      0.00      0.00         8
        1206       0.00      0.00      0.00         4
        1207       0.00      0.00      0.00         6
        1208       0.00      0.00      0.00         7
        1209       0.00      0.00      0.00         2
        1210       0.00      0.00      0.00         2
        1211       0.00      0.00      0.00         4
        1212       0.00      0.00      0.00         3
        1213       0.00      0.00      0.00         7
        1214       0.00      0.00      0.00         3
        1215       0.00      0.00      0.00         4
        1216       0.00      0.00      0.00         1
        1217       0.00      0.00      0.00         7
        1218       0.00      0.00      0.00        11
        1219       0.00      0.00      0.00         1
        1221       0.00      0.00      0.00         2
        1222       0.00      0.00      0.00         7
        1223       0.00      0.00      0.00         5
        1224       0.00      0.00      0.00         4
        1225       0.00      0.00      0.00         5
        1226       0.00      0.00      0.00         3
        1227       0.00      0.00      0.00         3
        1228       0.00      0.00      0.00         4
        1229       0.00      0.00      0.00        16
        1230       0.00      0.00      0.00         1
        1231       0.00      0.00      0.00         8
        1232       0.00      0.00      0.00         4
        1233       0.00      0.00      0.00         3
        1234       0.00      0.00      0.00         6
        1236       0.00      0.00      0.00         2
        1238       0.00      0.00      0.00         7
        1239       0.00      0.00      0.00         7
        1240       0.00      0.00      0.00         2
        1241       0.00      0.00      0.00         6
        1242       0.00      0.00      0.00         5
        1243       0.00      0.00      0.00         3
        1244       0.00      0.00      0.00         9
        1245       0.00      0.00      0.00         5
        1246       0.00      0.00      0.00         3
        1247       0.00      0.00      0.00         4
        1248       0.00      0.00      0.00         7
        1249       0.00      0.00      0.00         3
        1250       0.00      0.00      0.00         4
        1251       0.00      0.00      0.00         4
        1252       0.00      0.00      0.00         5
        1253       0.00      0.00      0.00         3
        1254       0.00      0.00      0.00         2
        1255       0.00      0.00      0.00         2
        1256       0.00      0.00      0.00         4
        1257       0.00      0.00      0.00         4
        1258       0.00      0.00      0.00         2
        1259       0.00      0.00      0.00         4
        1260       0.00      0.00      0.00         4
        1261       0.00      0.00      0.00         6
        1262       0.00      0.00      0.00         3
        1263       0.00      0.00      0.00         5
        1264       0.00      0.00      0.00         9
        1265       0.00      0.00      0.00        29
        1266       0.00      0.00      0.00         3
        1267       0.00      0.00      0.00         1
        1268       0.00      0.00      0.00         5
        1269       0.00      0.00      0.00         2
        1270       0.00      0.00      0.00         4
        1271       0.00      0.00      0.00         1
        1272       0.00      0.00      0.00         3
        1273       0.00      0.00      0.00         6
        1274       0.00      0.00      0.00         4
        1275       0.00      0.00      0.00        13
        1276       0.00      0.00      0.00         7
        1277       0.00      0.00      0.00         4
        1278       0.00      0.00      0.00         1
        1279       0.00      0.00      0.00         7
        1280       0.00      0.00      0.00        11
        1281       0.00      0.00      0.00         1
        1282       0.00      0.00      0.00         4
        1283       0.00      0.00      0.00         3
        1284       0.00      0.00      0.00         3
        1285       0.00      0.00      0.00         8
        1286       0.00      0.00      0.00         5
        1287       0.00      0.00      0.00         6
        1288       0.00      0.00      0.00         1
        1289       0.00      0.00      0.00         2
        1290       0.00      0.00      0.00         2
        1291       0.00      0.00      0.00         6
        1292       0.00      0.00      0.00         2
        1293       0.01      1.00      0.01        72
        1294       0.00      0.00      0.00         3
        1295       0.00      0.00      0.00        12
        1296       0.00      0.00      0.00         2
        1297       0.00      0.00      0.00         4
        1298       0.00      0.00      0.00         3
        1299       0.00      0.00      0.00         2
        1300       0.00      0.00      0.00         5
        1301       0.00      0.00      0.00         9
        1302       0.00      0.00      0.00         3
        1303       0.00      0.00      0.00         7
        1304       0.00      0.00      0.00        15
        1305       0.00      0.00      0.00         8
        1306       0.00      0.00      0.00         6
        1307       0.00      0.00      0.00        10
        1308       0.00      0.00      0.00         5
        1309       0.00      0.00      0.00         3
        1310       0.00      0.00      0.00         6
        1311       0.00      0.00      0.00         4
        1312       0.00      0.00      0.00         3
        1313       0.00      0.00      0.00         8
        1314       0.00      0.00      0.00         1
        1315       0.00      0.00      0.00         3
        1316       0.00      0.00      0.00        20
        1317       0.00      0.00      0.00         4
        1318       0.00      0.00      0.00         8
        1319       0.00      0.00      0.00         4
        1320       0.00      0.00      0.00         3
        1322       0.00      0.00      0.00         2
        1323       0.00      0.00      0.00         6
        1324       0.00      0.00      0.00        14
        1325       0.00      0.00      0.00         6
        1326       0.00      0.00      0.00         5
        1327       0.00      0.00      0.00        19
        1328       0.00      0.00      0.00         1
        1329       0.00      0.00      0.00         5
        1330       0.00      0.00      0.00         5
        1331       0.00      0.00      0.00         2
        1332       0.00      0.00      0.00         3
        1333       0.00      0.00      0.00         9
        1334       0.00      0.00      0.00         1
        1335       0.00      0.00      0.00         6
        1336       0.00      0.00      0.00        14
        1337       0.00      0.00      0.00        19
        1338       0.00      0.00      0.00         4
        1339       0.00      0.00      0.00         7
        1340       0.00      0.00      0.00         6
        1341       0.00      0.00      0.00         2
        1342       0.00      0.00      0.00         1
        1343       0.00      0.00      0.00        13
        1344       0.00      0.00      0.00         8
        1345       0.00      0.00      0.00         1
        1346       0.00      0.00      0.00        13
        1347       0.00      0.00      0.00         4
        1348       0.00      0.00      0.00         1
        1350       0.00      0.00      0.00         5
        1351       0.00      0.00      0.00         2
        1352       0.00      0.00      0.00         6
        1353       0.00      0.00      0.00         2
        1354       0.00      0.00      0.00        11
        1355       0.00      0.00      0.00         4
        1356       0.00      0.00      0.00         3
        1358       0.00      0.00      0.00         6
        1359       0.00      0.00      0.00         4
        1360       0.00      0.00      0.00         9
        1361       0.00      0.00      0.00         2
        1362       0.00      0.00      0.00         7
        1363       0.00      0.00      0.00         1
        1364       0.00      0.00      0.00         4
        1366       0.00      0.00      0.00        14
        1367       0.00      0.00      0.00         5
        1368       0.00      0.00      0.00        11
        1369       0.00      0.00      0.00         6
        1370       0.00      0.00      0.00         1
        1371       0.00      0.00      0.00         5
        1372       0.00      0.00      0.00         4
        1373       0.00      0.00      0.00        12
        1374       0.00      0.00      0.00        31
        1375       0.00      0.00      0.00         8
        1376       0.00      0.00      0.00         7
        1377       0.00      0.00      0.00         2
        1379       0.00      0.00      0.00         7
        1380       0.00      0.00      0.00         2
        1381       0.00      0.00      0.00         4
        1382       0.00      0.00      0.00         6
        1383       0.00      0.00      0.00         4
        1384       0.00      0.00      0.00        13
        1385       0.00      0.00      0.00         3
        1386       0.00      0.00      0.00         4
        1387       0.00      0.00      0.00         5
        1388       0.00      0.00      0.00        12
        1389       0.00      0.00      0.00         6
        1390       0.00      0.00      0.00         5
        1391       0.00      0.00      0.00         4
        1392       0.00      0.00      0.00        14
        1393       0.00      0.00      0.00         6
        1394       0.00      0.00      0.00         4
        1395       0.00      0.00      0.00         2
        1396       0.00      0.00      0.00         1
        1397       0.00      0.00      0.00         3
        1398       0.00      0.00      0.00         3
        1399       0.00      0.00      0.00         6
        1400       0.00      0.00      0.00         3
        1401       0.00      0.00      0.00         6
        1402       0.00      0.00      0.00         8
        1404       0.00      0.00      0.00         2
        1405       0.00      0.00      0.00         3
        1406       0.00      0.00      0.00         4
        1407       0.00      0.00      0.00         5
        1408       0.00      0.00      0.00         2
        1409       0.00      0.00      0.00         2
        1410       0.00      0.00      0.00         2
        1411       0.00      0.00      0.00         5
        1412       0.00      0.00      0.00         5
        1413       0.00      0.00      0.00         4
        1414       0.00      0.00      0.00         5
        1415       0.00      0.00      0.00         2
        1416       0.00      0.00      0.00         6
        1417       0.00      0.00      0.00         4
        1418       0.00      0.00      0.00         3
        1419       0.00      0.00      0.00         1
        1420       0.00      0.00      0.00         6
        1421       0.00      0.00      0.00         3
        1422       0.00      0.00      0.00         9
        1423       0.00      0.00      0.00         6
        1424       0.00      0.00      0.00         5
        1425       0.00      0.00      0.00         6
        1426       0.00      0.00      0.00         8
        1427       0.00      0.00      0.00         6
        1428       0.00      0.00      0.00         4
        1429       0.00      0.00      0.00         5
        1430       0.00      0.00      0.00         3
        1431       0.00      0.00      0.00         4
        1432       0.00      0.00      0.00         4
        1433       0.00      0.00      0.00         1
        1434       0.00      0.00      0.00         4
        1435       0.00      0.00      0.00         3
        1436       0.00      0.00      0.00         5
        1437       0.00      0.00      0.00         3
        1438       0.00      0.00      0.00         1
        1439       0.00      0.00      0.00         1
        1440       0.00      0.00      0.00         4
        1441       0.00      0.00      0.00         2
        1442       0.00      0.00      0.00         9
        1443       0.00      0.00      0.00         2
        1444       0.00      0.00      0.00         3
        1445       0.00      0.00      0.00         9
        1446       0.00      0.00      0.00         7
        1448       0.00      0.00      0.00         5
        1449       0.00      0.00      0.00        18
        1450       0.00      0.00      0.00         5
        1451       0.00      0.00      0.00         3
        1452       0.00      0.00      0.00         1
        1453       0.00      0.00      0.00         2
        1454       0.00      0.00      0.00         1
        1456       0.00      0.00      0.00         3
        1458       0.00      0.00      0.00         5
        1459       0.00      0.00      0.00         4
        1460       0.00      0.00      0.00         4
        1461       0.00      0.00      0.00         4
        1462       0.00      0.00      0.00         2
        1463       0.00      0.00      0.00         9
        1464       0.00      0.00      0.00        10
        1465       0.00      0.00      0.00         8
        1466       0.00      0.00      0.00         6
        1467       0.00      0.00      0.00         8
        1468       0.00      0.00      0.00         7
        1469       0.00      0.00      0.00         9
        1470       0.00      0.00      0.00         6
        1471       0.00      0.00      0.00         5
        1472       0.00      0.00      0.00         2
        1473       0.00      0.00      0.00         5
        1474       0.00      0.00      0.00         4
        1475       0.00      0.00      0.00         3
        1476       0.00      0.00      0.00         4
        1477       0.00      0.00      0.00         5
        1478       0.00      0.00      0.00         6
        1479       0.00      0.00      0.00         3
        1480       0.00      0.00      0.00         7
        1481       0.00      0.00      0.00         5
        1482       0.00      0.00      0.00         3
        1483       0.00      0.00      0.00         6
        1484       0.00      0.00      0.00         9
        1485       0.00      0.00      0.00        11
        1486       0.00      0.00      0.00         9
        1487       0.00      0.00      0.00         4
        1488       0.00      0.00      0.00         3
        1489       0.00      0.00      0.00         4
        1490       0.00      0.00      0.00         1
        1491       0.00      0.00      0.00         1
        1493       0.00      0.00      0.00        10
        1494       0.00      0.00      0.00         7
        1495       0.00      0.00      0.00         5
        1496       0.00      0.00      0.00         3
        1497       0.00      0.00      0.00         2
        1498       0.00      0.00      0.00         2
        1499       0.00      0.00      0.00        14
        1500       0.00      0.00      0.00         8
        1501       0.00      0.00      0.00        14
        1502       0.00      0.00      0.00         7
        1503       0.00      0.00      0.00         1
        1504       0.00      0.00      0.00         9
        1505       0.00      0.00      0.00         2
        1506       0.00      0.00      0.00         1
        1507       0.00      0.00      0.00         5
        1508       0.00      0.00      0.00         2
        1510       0.00      0.00      0.00         9
        1512       0.00      0.00      0.00         1
        1514       0.00      0.00      0.00         9
        1515       0.00      0.00      0.00         1
        1516       0.00      0.00      0.00         7
        1517       0.00      0.00      0.00         5
        1518       0.00      0.00      0.00         5
        1519       0.00      0.00      0.00         3
        1520       0.00      0.00      0.00         4
        1521       0.00      0.00      0.00         6
        1522       0.00      0.00      0.00         4
        1523       0.00      0.00      0.00         3
        1524       0.00      0.00      0.00         3
        1525       0.00      0.00      0.00         1
        1526       0.00      0.00      0.00         5
        1527       0.00      0.00      0.00         4
        1528       0.00      0.00      0.00         3
        1529       0.00      0.00      0.00         2
        1530       0.00      0.00      0.00         7
        1531       0.00      0.00      0.00         5
        1532       0.00      0.00      0.00         4
        1533       0.00      0.00      0.00         4
        1534       0.00      0.00      0.00         6
        1535       0.00      0.00      0.00         4
        1536       0.00      0.00      0.00         4
        1537       0.00      0.00      0.00         2
        1538       0.00      0.00      0.00         6
        1539       0.00      0.00      0.00         4
        1540       0.00      0.00      0.00         3
        1541       0.00      0.00      0.00         7
        1542       0.00      0.00      0.00         4
        1543       0.00      0.00      0.00         3
        1544       0.00      0.00      0.00         9
        1545       0.00      0.00      0.00        16
        1546       0.00      0.00      0.00         4
        1547       0.00      0.00      0.00         6
        1548       0.00      0.00      0.00         8
        1549       0.00      0.00      0.00         3
        1550       0.00      0.00      0.00         3
        1551       0.00      0.00      0.00         3
        1553       0.00      0.00      0.00         1
        1554       0.00      0.00      0.00         1
        1555       0.00      0.00      0.00         2
        1557       0.00      0.00      0.00         2
        1558       0.00      0.00      0.00         5
        1559       0.00      0.00      0.00         4
        1560       0.00      0.00      0.00         5
        1561       0.00      0.00      0.00        19
        1562       0.00      0.00      0.00         5
        1563       0.00      0.00      0.00        18
        1564       0.00      0.00      0.00         1
        1565       0.00      0.00      0.00         3
        1566       0.00      0.00      0.00         5
        1568       0.00      0.00      0.00         1
        1569       0.00      0.00      0.00         1
        1570       0.00      0.00      0.00         6
        1571       0.00      0.00      0.00         7
        1572       0.00      0.00      0.00         8
        1573       0.00      0.00      0.00         3
        1574       0.00      0.00      0.00         6
        1575       0.00      0.00      0.00         2
        1576       0.00      0.00      0.00        14
        1577       0.00      0.00      0.00         6
        1578       0.00      0.00      0.00         1
        1580       0.00      0.00      0.00         6
        1581       0.00      0.00      0.00         2
        1582       0.00      0.00      0.00         1
        1583       0.00      0.00      0.00         1
        1584       0.00      0.00      0.00         1
        1585       0.00      0.00      0.00         3
        1586       0.00      0.00      0.00         3
        1587       0.00      0.00      0.00         1
        1588       0.00      0.00      0.00         2
        1589       0.00      0.00      0.00         5
        1592       0.00      0.00      0.00         4
        1593       0.00      0.00      0.00        22
        1594       0.00      0.00      0.00         1
        1595       0.00      0.00      0.00         4
        1596       0.00      0.00      0.00         4
        1597       0.00      0.00      0.00         2
        1598       0.00      0.00      0.00         7
        1599       0.00      0.00      0.00         2
        1601       0.00      0.00      0.00         2
        1602       0.00      0.00      0.00         4
        1603       0.00      0.00      0.00         2
        1604       0.00      0.00      0.00        11
        1605       0.00      0.00      0.00         6
        1606       0.00      0.00      0.00         3
        1607       0.00      0.00      0.00         2
        1608       0.00      0.00      0.00         2
        1609       0.00      0.00      0.00         1
        1610       0.00      0.00      0.00         4
        1612       0.00      0.00      0.00         1
        1613       0.00      0.00      0.00         2
        1614       0.00      0.00      0.00         6
        1615       0.00      0.00      0.00         1
        1617       0.00      0.00      0.00         2
        1618       0.00      0.00      0.00         6
        1619       0.00      0.00      0.00         2
        1620       0.00      0.00      0.00        13
        1621       0.00      0.00      0.00         2
        1622       0.00      0.00      0.00         4
        1623       0.00      0.00      0.00         7
        1625       0.00      0.00      0.00         2
        1626       0.00      0.00      0.00         1
        1627       0.00      0.00      0.00         4
        1628       0.00      0.00      0.00         6
        1629       0.00      0.00      0.00         4
        1630       0.00      0.00      0.00         1
        1631       0.00      0.00      0.00         1
        1632       0.00      0.00      0.00         2
        1633       0.00      0.00      0.00         5
        1634       0.00      0.00      0.00         1
        1635       0.00      0.00      0.00        11
        1636       0.00      0.00      0.00         2
        1640       0.00      0.00      0.00         3
        1641       0.00      0.00      0.00         7
        1642       0.00      0.00      0.00         3
        1643       0.00      0.00      0.00         2
        1644       0.00      0.00      0.00        10
        1645       0.00      0.00      0.00         9
        1646       0.00      0.00      0.00         4
        1647       0.00      0.00      0.00         5
        1648       0.00      0.00      0.00         5
        1649       0.00      0.00      0.00         3
        1650       0.00      0.00      0.00         5
        1652       0.00      0.00      0.00         3
        1653       0.00      0.00      0.00         2
        1655       0.00      0.00      0.00         1
        1656       0.00      0.00      0.00         1
        1657       0.00      0.00      0.00         1
        1658       0.00      0.00      0.00         1
        1659       0.00      0.00      0.00        10
        1660       0.00      0.00      0.00         5
        1661       0.00      0.00      0.00         6
        1662       0.00      0.00      0.00        11
        1663       0.00      0.00      0.00         2
        1664       0.00      0.00      0.00         5
        1665       0.00      0.00      0.00         2
        1666       0.00      0.00      0.00         1
        1667       0.00      0.00      0.00         5
        1668       0.00      0.00      0.00         5
        1669       0.00      0.00      0.00         2
        1670       0.00      0.00      0.00         3
        1671       0.00      0.00      0.00         2
        1672       0.00      0.00      0.00         1
        1674       0.00      0.00      0.00         2
        1675       0.00      0.00      0.00         6
        1676       0.00      0.00      0.00         4
        1677       0.00      0.00      0.00         2
        1678       0.00      0.00      0.00         4
        1679       0.00      0.00      0.00         1
        1680       0.00      0.00      0.00         2
        1681       0.00      0.00      0.00         6
        1682       0.00      0.00      0.00         1
        1685       0.00      0.00      0.00         1
        1686       0.00      0.00      0.00         3
        1687       0.00      0.00      0.00         2
        1688       0.00      0.00      0.00         2
        1689       0.00      0.00      0.00         4
        1690       0.00      0.00      0.00         8
        1691       0.00      0.00      0.00         8
        1692       0.00      0.00      0.00         1
        1693       0.00      0.00      0.00         2
        1694       0.00      0.00      0.00         5
        1695       0.00      0.00      0.00         1
        1696       0.00      0.00      0.00         1
        1697       0.00      0.00      0.00         8
        1698       0.00      0.00      0.00         4
        1699       0.00      0.00      0.00         6
        1700       0.00      0.00      0.00         1
        1701       0.00      0.00      0.00         5
        1702       0.00      0.00      0.00        17
        1703       0.00      0.00      0.00         4
        1704       0.00      0.00      0.00         7
        1705       0.00      0.00      0.00         2
        1706       0.00      0.00      0.00         2
        1707       0.00      0.00      0.00         6
        1708       0.00      0.00      0.00         2
        1710       0.00      0.00      0.00         1
        1711       0.00      0.00      0.00         1
        1712       0.00      0.00      0.00         5
        1714       0.00      0.00      0.00         2
        1715       0.00      0.00      0.00         2
        1716       0.00      0.00      0.00         3
        1717       0.00      0.00      0.00         2
        1718       0.00      0.00      0.00         4
        1719       0.00      0.00      0.00         3
        1721       0.00      0.00      0.00         1
        1722       0.00      0.00      0.00         3
        1723       0.00      0.00      0.00         3
        1724       0.00      0.00      0.00         2
        1725       0.00      0.00      0.00         5
        1727       0.00      0.00      0.00         1
        1728       0.00      0.00      0.00         5
        1729       0.00      0.00      0.00         1
        1730       0.00      0.00      0.00         8
        1731       0.00      0.00      0.00         7
        1734       0.00      0.00      0.00         3
        1736       0.00      0.00      0.00         6
        1737       0.00      0.00      0.00         2
        1739       0.00      0.00      0.00         1
        1740       0.00      0.00      0.00         1
        1742       0.00      0.00      0.00         8
        1743       0.00      0.00      0.00         2
        1744       0.00      0.00      0.00         8
        1746       0.00      0.00      0.00         2
        1747       0.00      0.00      0.00         1
        1748       0.00      0.00      0.00         2
        1749       0.00      0.00      0.00         3
        1751       0.00      0.00      0.00         1
        1752       0.00      0.00      0.00         3
        1753       0.00      0.00      0.00         4
        1755       0.00      0.00      0.00         3
        1756       0.00      0.00      0.00         3
        1757       0.00      0.00      0.00         3
        1758       0.00      0.00      0.00         3
        1759       0.00      0.00      0.00        11
        1760       0.00      0.00      0.00        13
        1761       0.00      0.00      0.00         1
        1763       0.00      0.00      0.00         1
        1764       0.00      0.00      0.00         5
        1766       0.00      0.00      0.00        10
        1767       0.00      0.00      0.00         5
        1768       0.00      0.00      0.00         2
        1770       0.00      0.00      0.00         4
        1771       0.00      0.00      0.00         2
        1772       0.00      0.00      0.00         2
        1773       0.00      0.00      0.00         2
        1774       0.00      0.00      0.00         4
        1775       0.00      0.00      0.00         7
        1776       0.00      0.00      0.00         1
        1777       0.00      0.00      0.00         1
        1778       0.00      0.00      0.00         3
        1779       0.00      0.00      0.00         3
        1780       0.00      0.00      0.00         1
        1781       0.00      0.00      0.00         3
        1783       0.00      0.00      0.00         2
        1784       0.00      0.00      0.00         3
        1785       0.00      0.00      0.00         3
        1786       0.00      0.00      0.00         1
        1787       0.00      0.00      0.00         4
        1788       0.00      0.00      0.00         3
        1790       0.00      0.00      0.00        12
        1791       0.00      0.00      0.00         1
        1792       0.00      0.00      0.00         1
        1793       0.00      0.00      0.00         2
        1794       0.00      0.00      0.00         5
        1795       0.00      0.00      0.00        27
        1796       0.00      0.00      0.00         4
        1798       0.00      0.00      0.00         1
        1799       0.00      0.00      0.00         2
        1801       0.00      0.00      0.00         1
        1802       0.00      0.00      0.00         3
        1803       0.00      0.00      0.00         7
        1804       0.00      0.00      0.00         1
        1805       0.00      0.00      0.00         3
        1806       0.00      0.00      0.00         9
        1807       0.00      0.00      0.00         5
        1808       0.00      0.00      0.00         1
        1809       0.00      0.00      0.00         2
        1810       0.00      0.00      0.00         1
        1811       0.00      0.00      0.00         5
        1813       0.00      0.00      0.00         8
        1814       0.00      0.00      0.00         5
        1815       0.00      0.00      0.00         3
        1816       0.00      0.00      0.00         4
        1818       0.00      0.00      0.00         5
        1819       0.00      0.00      0.00         2
        1820       0.00      0.00      0.00         1
        1821       0.00      0.00      0.00         4
        1823       0.00      0.00      0.00         6
        1824       0.00      0.00      0.00         3
        1825       0.00      0.00      0.00         3
        1826       0.00      0.00      0.00         1
        1828       0.00      0.00      0.00         2
        1829       0.00      0.00      0.00         3
        1832       0.00      0.00      0.00         1
        1833       0.00      0.00      0.00         2
        1834       0.00      0.00      0.00         3
        1836       0.00      0.00      0.00         4
        1837       0.00      0.00      0.00         2
        1838       0.00      0.00      0.00         3
        1839       0.00      0.00      0.00         4
        1840       0.00      0.00      0.00         7
        1841       0.00      0.00      0.00         7
        1844       0.00      0.00      0.00         1
        1845       0.00      0.00      0.00         3
        1846       0.00      0.00      0.00         1
        1847       0.00      0.00      0.00         1
        1848       0.00      0.00      0.00         1
        1850       0.00      0.00      0.00         1
        1853       0.00      0.00      0.00        13
        1854       0.00      0.00      0.00        23
        1855       0.00      0.00      0.00         1
        1856       0.00      0.00      0.00         3
        1857       0.00      0.00      0.00         1
        1858       0.00      0.00      0.00         2
        1859       0.00      0.00      0.00         3
        1860       0.00      0.00      0.00         1
        1861       0.00      0.00      0.00         6
        1862       0.00      0.00      0.00         1
        1863       0.00      0.00      0.00         3
        1864       0.00      0.00      0.00         2
        1865       0.00      0.00      0.00         4
        1866       0.00      0.00      0.00         3
        1867       0.00      0.00      0.00         5
        1868       0.00      0.00      0.00         3
        1869       0.00      0.00      0.00         5
        1870       0.00      0.00      0.00         4
        1871       0.00      0.00      0.00         3
        1872       0.00      0.00      0.00         8
        1874       0.00      0.00      0.00         1
        1875       0.00      0.00      0.00         5
        1876       0.00      0.00      0.00         3
        1877       0.00      0.00      0.00         3
        1878       0.00      0.00      0.00         6
        1879       0.00      0.00      0.00         8
        1880       0.00      0.00      0.00         1
        1881       0.00      0.00      0.00         1
        1882       0.00      0.00      0.00        26
        1883       0.00      0.00      0.00         1
        1884       0.00      0.00      0.00         5
        1885       0.00      0.00      0.00         3
        1889       0.00      0.00      0.00         1
        1892       0.00      0.00      0.00         2
        1893       0.00      0.00      0.00         1
        1894       0.00      0.00      0.00         2
        1896       0.00      0.00      0.00         3
        1898       0.00      0.00      0.00         1
        1899       0.00      0.00      0.00         1
        1900       0.00      0.00      0.00         6
        1901       0.00      0.00      0.00         9
        1902       0.00      0.00      0.00         4
        1906       0.00      0.00      0.00        11
        1907       0.00      0.00      0.00         2
        1908       0.00      0.00      0.00         1
        1909       0.00      0.00      0.00         2
        1910       0.00      0.00      0.00         4
        1911       0.00      0.00      0.00         4
        1912       0.00      0.00      0.00         2
        1913       0.00      0.00      0.00         4
        1914       0.00      0.00      0.00         2
        1917       0.00      0.00      0.00         6
        1918       0.00      0.00      0.00         6
        1919       0.00      0.00      0.00         1
        1920       0.00      0.00      0.00         6
        1921       0.00      0.00      0.00         1
        1922       0.00      0.00      0.00         1
        1923       0.00      0.00      0.00         1
        1924       0.00      0.00      0.00         4
        1925       0.00      0.00      0.00         1
        1926       0.00      0.00      0.00         9
        1927       0.00      0.00      0.00         2
        1928       0.00      0.00      0.00         3
        1929       0.00      0.00      0.00         3
        1931       0.00      0.00      0.00         3
        1932       0.00      0.00      0.00         1
        1933       0.00      0.00      0.00         8
        1934       0.00      0.00      0.00         2
        1935       0.00      0.00      0.00         3
        1936       0.00      0.00      0.00         3
        1937       0.00      0.00      0.00         9
        1938       0.00      0.00      0.00         2
        1939       0.00      0.00      0.00         6
        1940       0.00      0.00      0.00         2
        1941       0.00      0.00      0.00         1
        1942       0.00      0.00      0.00         1
        1943       0.00      0.00      0.00         2
        1944       0.00      0.00      0.00         4
        1945       0.00      0.00      0.00         1
        1948       0.00      0.00      0.00         4
        1950       0.00      0.00      0.00         3
        1951       0.00      0.00      0.00         7
        1952       0.00      0.00      0.00         1
        1953       0.00      0.00      0.00         1
        1954       0.00      0.00      0.00         3
        1955       0.00      0.00      0.00         3
        1956       0.00      0.00      0.00         1
        1957       0.00      0.00      0.00         8
        1958       0.00      0.00      0.00         5
        1961       0.00      0.00      0.00         4
        1963       0.00      0.00      0.00         2
        1964       0.00      0.00      0.00         2
        1965       0.00      0.00      0.00         5
        1966       0.00      0.00      0.00         1
        1967       0.00      0.00      0.00         1
        1969       0.00      0.00      0.00         3
        1970       0.00      0.00      0.00         1
        1971       0.00      0.00      0.00         7
        1973       0.00      0.00      0.00         3
        1974       0.00      0.00      0.00         1
        1975       0.00      0.00      0.00         1
        1976       0.00      0.00      0.00         5
        1977       0.00      0.00      0.00         1
        1978       0.00      0.00      0.00         1
        1979       0.00      0.00      0.00         3
        1980       0.00      0.00      0.00         1
        1981       0.00      0.00      0.00         3
        1983       0.00      0.00      0.00         4
        1984       0.00      0.00      0.00         1
        1985       0.00      0.00      0.00         1
        1986       0.00      0.00      0.00         2
        1987       0.00      0.00      0.00         3
        1988       0.00      0.00      0.00         4
        1989       0.00      0.00      0.00        11
        1990       0.00      0.00      0.00         2
        1991       0.00      0.00      0.00         4
        1992       0.00      0.00      0.00         6
        1993       0.00      0.00      0.00         3
        1994       0.00      0.00      0.00         3
        1995       0.00      0.00      0.00         5
        1996       0.00      0.00      0.00         1
        1998       0.00      0.00      0.00         1
        1999       0.00      0.00      0.00         3
        2000       0.00      0.00      0.00         6
        2001       0.00      0.00      0.00         7
        2002       0.00      0.00      0.00         8
        2003       0.00      0.00      0.00         8
        2006       0.00      0.00      0.00         1
        2008       0.00      0.00      0.00        19
        2009       0.00      0.00      0.00         7
        2010       0.00      0.00      0.00         1
        2011       0.00      0.00      0.00         5
        2012       0.00      0.00      0.00         1
        2014       0.00      0.00      0.00         6
        2015       0.00      0.00      0.00         5
        2016       0.00      0.00      0.00         1
        2017       0.00      0.00      0.00         2
        2018       0.00      0.00      0.00         1
        2019       0.00      0.00      0.00         2
        2020       0.00      0.00      0.00        10
        2021       0.00      0.00      0.00         1
        2022       0.00      0.00      0.00         2
        2023       0.00      0.00      0.00         5
        2024       0.00      0.00      0.00         1
        2025       0.00      0.00      0.00         1
        2026       0.00      0.00      0.00         4
        2028       0.00      0.00      0.00         5
        2029       0.00      0.00      0.00         2
        2030       0.00      0.00      0.00         4
        2031       0.00      0.00      0.00         2
        2032       0.00      0.00      0.00         1
        2033       0.00      0.00      0.00        11
        2034       0.00      0.00      0.00         4
        2035       0.00      0.00      0.00         2
        2036       0.00      0.00      0.00         6
        2037       0.00      0.00      0.00         6
        2039       0.00      0.00      0.00         2
        2040       0.00      0.00      0.00         5
        2041       0.00      0.00      0.00         3
        2044       0.00      0.00      0.00         5
        2045       0.00      0.00      0.00         2
        2046       0.00      0.00      0.00         6
        2047       0.00      0.00      0.00         2
        2048       0.00      0.00      0.00        22
        2049       0.00      0.00      0.00         4
        2050       0.00      0.00      0.00         7
        2051       0.00      0.00      0.00         1
        2053       0.00      0.00      0.00         2
        2055       0.00      0.00      0.00         2
        2056       0.00      0.00      0.00         1
        2057       0.00      0.00      0.00         6
        2058       0.00      0.00      0.00         3
        2060       0.00      0.00      0.00         1
        2061       0.00      0.00      0.00         6
        2062       0.00      0.00      0.00         2
        2063       0.00      0.00      0.00         1
        2064       0.00      0.00      0.00         3
        2067       0.00      0.00      0.00         1
        2068       0.00      0.00      0.00         2
        2069       0.00      0.00      0.00         4
        2070       0.00      0.00      0.00         2
        2072       0.00      0.00      0.00         4
        2073       0.00      0.00      0.00         2
        2074       0.00      0.00      0.00         5
        2076       0.00      0.00      0.00         2
        2077       0.00      0.00      0.00         5
        2080       0.00      0.00      0.00         4
        2081       0.00      0.00      0.00         1
        2082       0.00      0.00      0.00         1
        2083       0.00      0.00      0.00         4
        2084       0.00      0.00      0.00         6
        2085       0.00      0.00      0.00         7
        2086       0.00      0.00      0.00         3
        2087       0.00      0.00      0.00         1
        2088       0.00      0.00      0.00         2
        2089       0.00      0.00      0.00         1
        2090       0.00      0.00      0.00         5
        2091       0.00      0.00      0.00         1
        2092       0.00      0.00      0.00         3
        2093       0.00      0.00      0.00         8
        2094       0.00      0.00      0.00         6
        2095       0.00      0.00      0.00         3
        2096       0.00      0.00      0.00         1
        2097       0.00      0.00      0.00         1
        2098       0.00      0.00      0.00         7
        2099       0.00      0.00      0.00         2
        2100       0.00      0.00      0.00        13
        2101       0.00      0.00      0.00         3
        2102       0.00      0.00      0.00         2
        2103       0.00      0.00      0.00         4
        2105       0.00      0.00      0.00         4
        2106       0.00      0.00      0.00         1
        2107       0.00      0.00      0.00         1
        2108       0.00      0.00      0.00         3
        2109       0.00      0.00      0.00         2
        2110       0.00      0.00      0.00         4
        2111       0.00      0.00      0.00         3
        2112       0.00      0.00      0.00         3
        2113       0.00      0.00      0.00         6
        2114       0.00      0.00      0.00         3
        2115       0.00      0.00      0.00         1
        2116       0.00      0.00      0.00         5
        2117       0.00      0.00      0.00         8
        2118       0.00      0.00      0.00         1
        2119       0.00      0.00      0.00         7
        2120       0.00      0.00      0.00         4
        2121       0.00      0.00      0.00         3
        2123       0.00      0.00      0.00        10
        2124       0.00      0.00      0.00         1
        2125       0.00      0.00      0.00         1
        2126       0.00      0.00      0.00         1
        2128       0.00      0.00      0.00         2
        2129       0.00      0.00      0.00         5
        2130       0.00      0.00      0.00         8
        2131       0.00      0.00      0.00         4
        2132       0.00      0.00      0.00         3
        2133       0.00      0.00      0.00         4
        2134       0.00      0.00      0.00         8
        2135       0.00      0.00      0.00         6
        2136       0.00      0.00      0.00         2
        2137       0.00      0.00      0.00         1
        2138       0.00      0.00      0.00         3
        2139       0.00      0.00      0.00         3
        2140       0.00      0.00      0.00         4
        2141       0.00      0.00      0.00         3
        2142       0.00      0.00      0.00         1
        2143       0.00      0.00      0.00         3
        2144       0.00      0.00      0.00         1
        2146       0.00      0.00      0.00         1
        2147       0.00      0.00      0.00         2
        2148       0.00      0.00      0.00         2
        2149       0.00      0.00      0.00         4
        2150       0.00      0.00      0.00         3
        2153       0.00      0.00      0.00         5
        2154       0.00      0.00      0.00         1
        2156       0.00      0.00      0.00         1
        2159       0.00      0.00      0.00         4
        2162       0.00      0.00      0.00         3
        2163       0.00      0.00      0.00         4
        2164       0.00      0.00      0.00         4
        2165       0.00      0.00      0.00         2
        2166       0.00      0.00      0.00         1
        2168       0.00      0.00      0.00         8
        2169       0.00      0.00      0.00         9
        2172       0.00      0.00      0.00         6
        2173       0.00      0.00      0.00         2
        2174       0.00      0.00      0.00         1
        2175       0.00      0.00      0.00         3
        2177       0.00      0.00      0.00         2
        2178       0.00      0.00      0.00         4
        2179       0.00      0.00      0.00         4
        2180       0.00      0.00      0.00         5
        2181       0.00      0.00      0.00         7
        2182       0.00      0.00      0.00         6
        2183       0.00      0.00      0.00         6
        2184       0.00      0.00      0.00         1
        2185       0.00      0.00      0.00         2
        2187       0.00      0.00      0.00         1
        2188       0.00      0.00      0.00         4
        2189       0.00      0.00      0.00         1
        2190       0.00      0.00      0.00         1
        2191       0.00      0.00      0.00         1
        2192       0.00      0.00      0.00         2
        2196       0.00      0.00      0.00         1
        2197       0.00      0.00      0.00         5
        2198       0.00      0.00      0.00         2
        2199       0.00      0.00      0.00         3
        2200       0.00      0.00      0.00         4
        2201       0.00      0.00      0.00         3
        2205       0.00      0.00      0.00         1
        2206       0.00      0.00      0.00         2
        2207       0.00      0.00      0.00         3
        2209       0.00      0.00      0.00         1
        2210       0.00      0.00      0.00         3
        2215       0.00      0.00      0.00         3
        2216       0.00      0.00      0.00         1
        2217       0.00      0.00      0.00         5
        2218       0.00      0.00      0.00         1
        2219       0.00      0.00      0.00         4
        2221       0.00      0.00      0.00         5
        2222       0.00      0.00      0.00         1
        2224       0.00      0.00      0.00         2
        2225       0.00      0.00      0.00         1
        2226       0.00      0.00      0.00         3
        2227       0.00      0.00      0.00         1
        2229       0.00      0.00      0.00         2
        2230       0.00      0.00      0.00         5
        2231       0.00      0.00      0.00         1
        2233       0.00      0.00      0.00         1
        2236       0.00      0.00      0.00         3
        2237       0.00      0.00      0.00         2
        2238       0.00      0.00      0.00         1
        2239       0.00      0.00      0.00         1
        2240       0.00      0.00      0.00         2
        2242       0.00      0.00      0.00         1
        2244       0.00      0.00      0.00         2
        2245       0.00      0.00      0.00         4
        2246       0.00      0.00      0.00         1
        2247       0.00      0.00      0.00         3
        2248       0.00      0.00      0.00         2
        2249       0.00      0.00      0.00         2
        2250       0.00      0.00      0.00         4
        2251       0.00      0.00      0.00         2
        2253       0.00      0.00      0.00         4
        2254       0.00      0.00      0.00         1
        2255       0.00      0.00      0.00         1
        2256       0.00      0.00      0.00         1
        2257       0.00      0.00      0.00         2
        2258       0.00      0.00      0.00         4
        2259       0.00      0.00      0.00         3
        2260       0.00      0.00      0.00         1
        2262       0.00      0.00      0.00         6
        2263       0.00      0.00      0.00         4
        2264       0.00      0.00      0.00         2
        2265       0.00      0.00      0.00         1
        2267       0.00      0.00      0.00         4
        2268       0.00      0.00      0.00         6
        2272       0.00      0.00      0.00         1
        2273       0.00      0.00      0.00         1
        2274       0.00      0.00      0.00         6
        2275       0.00      0.00      0.00         3
        2276       0.00      0.00      0.00        10
        2277       0.00      0.00      0.00         5
        2279       0.00      0.00      0.00         1
        2280       0.00      0.00      0.00         2
        2282       0.00      0.00      0.00         3
        2283       0.00      0.00      0.00         2
        2284       0.00      0.00      0.00         2
        2286       0.00      0.00      0.00         1
        2287       0.00      0.00      0.00         3
        2288       0.00      0.00      0.00         1
        2290       0.00      0.00      0.00         2
        2291       0.00      0.00      0.00         1
        2292       0.00      0.00      0.00         5
        2293       0.00      0.00      0.00         6
        2294       0.00      0.00      0.00         4
        2295       0.00      0.00      0.00         5
        2297       0.00      0.00      0.00         1
        2300       0.00      0.00      0.00         1
        2301       0.00      0.00      0.00         1
        2303       0.00      0.00      0.00         5
        2305       0.00      0.00      0.00         3
        2306       0.00      0.00      0.00         3
        2307       0.00      0.00      0.00         7
        2308       0.00      0.00      0.00         1
        2309       0.00      0.00      0.00         2
        2311       0.00      0.00      0.00         1
        2313       0.00      0.00      0.00         2
        2314       0.00      0.00      0.00         4
        2315       0.00      0.00      0.00         5
        2316       0.00      0.00      0.00         4
        2318       0.00      0.00      0.00         3
        2319       0.00      0.00      0.00         1
        2321       0.00      0.00      0.00        14
        2322       0.00      0.00      0.00        12
        2323       0.00      0.00      0.00         3
        2324       0.00      0.00      0.00         5
        2325       0.00      0.00      0.00         1
        2326       0.00      0.00      0.00         1
        2327       0.00      0.00      0.00         5
        2328       0.00      0.00      0.00         2
        2329       0.00      0.00      0.00         1
        2331       0.00      0.00      0.00         2
        2332       0.00      0.00      0.00         2
        2333       0.00      0.00      0.00         3
        2334       0.00      0.00      0.00         3
        2341       0.00      0.00      0.00         2
        2342       0.00      0.00      0.00         1
        2343       0.00      0.00      0.00         3
        2344       0.00      0.00      0.00         2
        2345       0.00      0.00      0.00         1
        2347       0.00      0.00      0.00         2
        2349       0.00      0.00      0.00         1
        2350       0.00      0.00      0.00         2
        2351       0.00      0.00      0.00         1
        2352       0.00      0.00      0.00         3
        2355       0.00      0.00      0.00         1
        2356       0.00      0.00      0.00         3
        2357       0.00      0.00      0.00         4
        2361       0.00      0.00      0.00         1
        2362       0.00      0.00      0.00         2
        2363       0.00      0.00      0.00         2
        2364       0.00      0.00      0.00         4
        2365       0.00      0.00      0.00         1
        2368       0.00      0.00      0.00         1
        2369       0.00      0.00      0.00         2
        2371       0.00      0.00      0.00         2
        2372       0.00      0.00      0.00         4
        2373       0.00      0.00      0.00         2
        2374       0.00      0.00      0.00         2
        2375       0.00      0.00      0.00         7
        2376       0.00      0.00      0.00         1
        2377       0.00      0.00      0.00         2
        2379       0.00      0.00      0.00         1
        2380       0.00      0.00      0.00         2
        2381       0.00      0.00      0.00         3
        2382       0.00      0.00      0.00         2
        2384       0.00      0.00      0.00         3
        2386       0.00      0.00      0.00         5
        2388       0.00      0.00      0.00         1
        2389       0.00      0.00      0.00         5
        2390       0.00      0.00      0.00         6
        2391       0.00      0.00      0.00         4
        2392       0.00      0.00      0.00         3
        2393       0.00      0.00      0.00         4
        2394       0.00      0.00      0.00         6
        2395       0.00      0.00      0.00         5
        2396       0.00      0.00      0.00         2
        2397       0.00      0.00      0.00         2
        2398       0.00      0.00      0.00         2
        2400       0.00      0.00      0.00         3
        2401       0.00      0.00      0.00         2
        2402       0.00      0.00      0.00         6
        2403       0.00      0.00      0.00        16
        2404       0.00      0.00      0.00         2
        2406       0.00      0.00      0.00         3
        2407       0.00      0.00      0.00         3
        2410       0.00      0.00      0.00        18
        2411       0.00      0.00      0.00         2
        2413       0.00      0.00      0.00         1
        2415       0.00      0.00      0.00         1
        2416       0.00      0.00      0.00         2
        2418       0.00      0.00      0.00         3
        2420       0.00      0.00      0.00         4
        2421       0.00      0.00      0.00         1
        2423       0.00      0.00      0.00         9
        2424       0.00      0.00      0.00         3
        2425       0.00      0.00      0.00         2
        2427       0.00      0.00      0.00         1
        2428       0.00      0.00      0.00         1
        2429       0.00      0.00      0.00         4
        2430       0.00      0.00      0.00         1
        2431       0.00      0.00      0.00         2
        2432       0.00      0.00      0.00         1
        2433       0.00      0.00      0.00         3
        2434       0.00      0.00      0.00         5
        2435       0.00      0.00      0.00         1
        2436       0.00      0.00      0.00         3
        2437       0.00      0.00      0.00         3
        2439       0.00      0.00      0.00         1
        2440       0.00      0.00      0.00         1
        2441       0.00      0.00      0.00         1
        2442       0.00      0.00      0.00         5
        2444       0.00      0.00      0.00         1
        2445       0.00      0.00      0.00         3
        2446       0.00      0.00      0.00         2
        2447       0.00      0.00      0.00         1
        2448       0.00      0.00      0.00         2
        2449       0.00      0.00      0.00         3
        2451       0.00      0.00      0.00         1
        2452       0.00      0.00      0.00         5
        2454       0.00      0.00      0.00         1
        2455       0.00      0.00      0.00         3
        2456       0.00      0.00      0.00         4
        2457       0.00      0.00      0.00         1
        2459       0.00      0.00      0.00         5
        2460       0.00      0.00      0.00         5
        2461       0.00      0.00      0.00         1
        2462       0.00      0.00      0.00         3
        2463       0.00      0.00      0.00         3
        2465       0.00      0.00      0.00         4
        2466       0.00      0.00      0.00         2
        2467       0.00      0.00      0.00         3
        2468       0.00      0.00      0.00         1
        2469       0.00      0.00      0.00         1
        2470       0.00      0.00      0.00        21
        2471       0.00      0.00      0.00         6
        2472       0.00      0.00      0.00         1
        2473       0.00      0.00      0.00         8
        2475       0.00      0.00      0.00         3
        2480       0.00      0.00      0.00         3
        2481       0.00      0.00      0.00         4
        2482       0.00      0.00      0.00         1
        2483       0.00      0.00      0.00         1
        2484       0.00      0.00      0.00         1
        2486       0.00      0.00      0.00         4
        2489       0.00      0.00      0.00         4
        2491       0.00      0.00      0.00         8
        2492       0.00      0.00      0.00         1
        2493       0.00      0.00      0.00         2
        2496       0.00      0.00      0.00         3
        2497       0.00      0.00      0.00         1
        2500       0.00      0.00      0.00         1
        2501       0.00      0.00      0.00         2
        2502       0.00      0.00      0.00         2
        2504       0.00      0.00      0.00         3
        2505       0.00      0.00      0.00         1
        2506       0.00      0.00      0.00         2
        2508       0.00      0.00      0.00         3
        2512       0.00      0.00      0.00         3
        2513       0.00      0.00      0.00         1
        2515       0.00      0.00      0.00         2
        2516       0.00      0.00      0.00         2
        2519       0.00      0.00      0.00         1
        2520       0.00      0.00      0.00         1
        2521       0.00      0.00      0.00         7
        2522       0.00      0.00      0.00         1
        2523       0.00      0.00      0.00         1
        2524       0.00      0.00      0.00         1
        2525       0.00      0.00      0.00         3
        2528       0.00      0.00      0.00         1
        2529       0.00      0.00      0.00        10
        2530       0.00      0.00      0.00         3
        2532       0.00      0.00      0.00         2
        2533       0.00      0.00      0.00         5
        2535       0.00      0.00      0.00         2
        2536       0.00      0.00      0.00         2
        2537       0.00      0.00      0.00         2
        2538       0.00      0.00      0.00         1
        2539       0.00      0.00      0.00         3
        2540       0.00      0.00      0.00         3
        2542       0.00      0.00      0.00         7
        2543       0.00      0.00      0.00         8
        2545       0.00      0.00      0.00         1
        2547       0.00      0.00      0.00         1
        2549       0.00      0.00      0.00         7
        2550       0.00      0.00      0.00         6
        2551       0.00      0.00      0.00         2
        2552       0.00      0.00      0.00         1
        2553       0.00      0.00      0.00         6
        2555       0.00      0.00      0.00         1
        2558       0.00      0.00      0.00         7
        2559       0.00      0.00      0.00         1
        2560       0.00      0.00      0.00         7
        2561       0.00      0.00      0.00         2
        2562       0.00      0.00      0.00         1
        2563       0.00      0.00      0.00         1
        2566       0.00      0.00      0.00         1
        2567       0.00      0.00      0.00         4
        2568       0.00      0.00      0.00         3
        2571       0.00      0.00      0.00         1
        2572       0.00      0.00      0.00         3
        2574       0.00      0.00      0.00         1
        2575       0.00      0.00      0.00         2
        2577       0.00      0.00      0.00         1
        2578       0.00      0.00      0.00         1
        2579       0.00      0.00      0.00         3
        2581       0.00      0.00      0.00         5
        2582       0.00      0.00      0.00         2
        2583       0.00      0.00      0.00         2
        2584       0.00      0.00      0.00         4
        2585       0.00      0.00      0.00         1
        2587       0.00      0.00      0.00         3
        2588       0.00      0.00      0.00         7
        2589       0.00      0.00      0.00         2
        2590       0.00      0.00      0.00         3
        2592       0.00      0.00      0.00         2
        2593       0.00      0.00      0.00         3
        2594       0.00      0.00      0.00         2
        2595       0.00      0.00      0.00         6
        2596       0.00      0.00      0.00         1
        2597       0.00      0.00      0.00         6
        2598       0.00      0.00      0.00         1
        2601       0.00      0.00      0.00         2
        2603       0.00      0.00      0.00         2
        2605       0.00      0.00      0.00         6
        2606       0.00      0.00      0.00         5
        2607       0.00      0.00      0.00         1
        2609       0.00      0.00      0.00         2
        2610       0.00      0.00      0.00         5
        2612       0.00      0.00      0.00         2
        2613       0.00      0.00      0.00         7
        2614       0.00      0.00      0.00         1
        2615       0.00      0.00      0.00         8
        2616       0.00      0.00      0.00         2
        2617       0.00      0.00      0.00         1
        2618       0.00      0.00      0.00         1
        2619       0.00      0.00      0.00         1
        2620       0.00      0.00      0.00         2
        2622       0.00      0.00      0.00         6
        2623       0.00      0.00      0.00         4
        2624       0.00      0.00      0.00         3
        2625       0.00      0.00      0.00         2
        2626       0.00      0.00      0.00         6
        2627       0.00      0.00      0.00         2
        2631       0.00      0.00      0.00         3
        2632       0.00      0.00      0.00         1
        2635       0.00      0.00      0.00         3
        2636       0.00      0.00      0.00         2
        2637       0.00      0.00      0.00         1
        2638       0.00      0.00      0.00         3
        2640       0.00      0.00      0.00         1
        2641       0.00      0.00      0.00         1
        2643       0.00      0.00      0.00         4
        2646       0.00      0.00      0.00         1
        2648       0.00      0.00      0.00         2
        2652       0.00      0.00      0.00         1
        2653       0.00      0.00      0.00         1
        2655       0.00      0.00      0.00         1
        2657       0.00      0.00      0.00        10
        2658       0.00      0.00      0.00         2
        2663       0.00      0.00      0.00         2
        2664       0.00      0.00      0.00         1
        2665       0.00      0.00      0.00         1
        2666       0.00      0.00      0.00         5
        2667       0.00      0.00      0.00         3
        2669       0.00      0.00      0.00         2
        2671       0.00      0.00      0.00         1
        2674       0.00      0.00      0.00         2
        2675       0.00      0.00      0.00         1
        2678       0.00      0.00      0.00         1
        2681       0.00      0.00      0.00         1
        2684       0.00      0.00      0.00         1
        2685       0.00      0.00      0.00         1
        2686       0.00      0.00      0.00         3
        2687       0.00      0.00      0.00         2
        2690       0.00      0.00      0.00         3
        2691       0.00      0.00      0.00         2
        2692       0.00      0.00      0.00         1
        2693       0.00      0.00      0.00         1
        2694       0.00      0.00      0.00         2
        2695       0.00      0.00      0.00         1
        2697       0.00      0.00      0.00         7
        2698       0.00      0.00      0.00         2
        2699       0.00      0.00      0.00         2
        2700       0.00      0.00      0.00         1
        2701       0.00      0.00      0.00         1
        2703       0.00      0.00      0.00         2
        2704       0.00      0.00      0.00         2
        2705       0.00      0.00      0.00         2
        2708       0.00      0.00      0.00         1
        2710       0.00      0.00      0.00         2
        2711       0.00      0.00      0.00         2
        2712       0.00      0.00      0.00         1
        2713       0.00      0.00      0.00         1
        2714       0.00      0.00      0.00         1
        2715       0.00      0.00      0.00         5
        2716       0.00      0.00      0.00         1
        2717       0.00      0.00      0.00         1
        2718       0.00      0.00      0.00         1
        2720       0.00      0.00      0.00         1
        2722       0.00      0.00      0.00         2
        2724       0.00      0.00      0.00         2
        2725       0.00      0.00      0.00         1
        2726       0.00      0.00      0.00         1
        2727       0.00      0.00      0.00         1
        2732       0.00      0.00      0.00         1
        2735       0.00      0.00      0.00         1
        2736       0.00      0.00      0.00         4
        2737       0.00      0.00      0.00         5
        2738       0.00      0.00      0.00         2
        2740       0.00      0.00      0.00         4
        2741       0.00      0.00      0.00         1
        2742       0.00      0.00      0.00         2
        2744       0.00      0.00      0.00         2
        2745       0.00      0.00      0.00         2
        2746       0.00      0.00      0.00         4
        2748       0.00      0.00      0.00         6
        2750       0.00      0.00      0.00         4
        2753       0.00      0.00      0.00         9
        2754       0.00      0.00      0.00         7
        2755       0.00      0.00      0.00         1
        2756       0.00      0.00      0.00         1
        2757       0.00      0.00      0.00         5
        2759       0.00      0.00      0.00         2
        2761       0.00      0.00      0.00         1
        2762       0.00      0.00      0.00         7
        2764       0.00      0.00      0.00         2
        2766       0.00      0.00      0.00         1
        2768       0.00      0.00      0.00         1
        2769       0.00      0.00      0.00         2
        2770       0.00      0.00      0.00         3
        2771       0.00      0.00      0.00         1
        2772       0.00      0.00      0.00         2
        2773       0.00      0.00      0.00         2
        2774       0.00      0.00      0.00         1
        2778       0.00      0.00      0.00         1
        2779       0.00      0.00      0.00         2
        2780       0.00      0.00      0.00         2
        2781       0.00      0.00      0.00         1
        2783       0.00      0.00      0.00         2
        2785       0.00      0.00      0.00         3
        2786       0.00      0.00      0.00         1
        2787       0.00      0.00      0.00         1
        2788       0.00      0.00      0.00         4
        2792       0.00      0.00      0.00         1
        2793       0.00      0.00      0.00         1
        2794       0.00      0.00      0.00         2
        2797       0.00      0.00      0.00         1
        2799       0.00      0.00      0.00         2
        2800       0.00      0.00      0.00         2
        2801       0.00      0.00      0.00         4
        2802       0.00      0.00      0.00         1
        2803       0.00      0.00      0.00         2
        2804       0.00      0.00      0.00         2
        2805       0.00      0.00      0.00         1
        2806       0.00      0.00      0.00         2
        2808       0.00      0.00      0.00         3
        2810       0.00      0.00      0.00         4
        2811       0.00      0.00      0.00         2
        2812       0.00      0.00      0.00         1
        2813       0.00      0.00      0.00         1
        2814       0.00      0.00      0.00         1
        2816       0.00      0.00      0.00         1
        2818       0.00      0.00      0.00         3
        2819       0.00      0.00      0.00         8
        2820       0.00      0.00      0.00         1
        2821       0.00      0.00      0.00         2
        2824       0.00      0.00      0.00         3
        2825       0.00      0.00      0.00         1
        2826       0.00      0.00      0.00         2
        2827       0.00      0.00      0.00         2
        2830       0.00      0.00      0.00         3
        2831       0.00      0.00      0.00         2
        2832       0.00      0.00      0.00         5
        2833       0.00      0.00      0.00         2
        2834       0.00      0.00      0.00         2
        2835       0.00      0.00      0.00        16
        2836       0.00      0.00      0.00         1
        2837       0.00      0.00      0.00         4
        2838       0.00      0.00      0.00         5
        2839       0.00      0.00      0.00         2
        2841       0.00      0.00      0.00         2
        2842       0.00      0.00      0.00         2
        2843       0.00      0.00      0.00         1
        2846       0.00      0.00      0.00         3
        2848       0.00      0.00      0.00         3
        2851       0.00      0.00      0.00         2
        2852       0.00      0.00      0.00         2
        2853       0.00      0.00      0.00         3
        2855       0.00      0.00      0.00         1
        2857       0.00      0.00      0.00         1
        2858       0.00      0.00      0.00         2
        2860       0.00      0.00      0.00         1
        2864       0.00      0.00      0.00         1
        2865       0.00      0.00      0.00         4
        2870       0.00      0.00      0.00         1
        2872       0.00      0.00      0.00         1
        2873       0.00      0.00      0.00         6
        2874       0.00      0.00      0.00         3
        2876       0.00      0.00      0.00         1
        2878       0.00      0.00      0.00         3
        2880       0.00      0.00      0.00         1
        2884       0.00      0.00      0.00         1
        2886       0.00      0.00      0.00         1
        2887       0.00      0.00      0.00         6
        2889       0.00      0.00      0.00         1
        2890       0.00      0.00      0.00         2
        2894       0.00      0.00      0.00         1
        2895       0.00      0.00      0.00         4
        2896       0.00      0.00      0.00         2
        2897       0.00      0.00      0.00         1
        2899       0.00      0.00      0.00         1
        2900       0.00      0.00      0.00         1
        2901       0.00      0.00      0.00         3
        2902       0.00      0.00      0.00         2
        2903       0.00      0.00      0.00         1
        2905       0.00      0.00      0.00         1
        2906       0.00      0.00      0.00         1
        2907       0.00      0.00      0.00         2
        2910       0.00      0.00      0.00         2
        2913       0.00      0.00      0.00         1
        2915       0.00      0.00      0.00         6
        2917       0.00      0.00      0.00         1
        2918       0.00      0.00      0.00         3
        2919       0.00      0.00      0.00         3
        2920       0.00      0.00      0.00         1
        2923       0.00      0.00      0.00         2
        2925       0.00      0.00      0.00         2
        2927       0.00      0.00      0.00         4
        2928       0.00      0.00      0.00         1
        2929       0.00      0.00      0.00         3
        2930       0.00      0.00      0.00         4
        2931       0.00      0.00      0.00         6
        2932       0.00      0.00      0.00         1
        2933       0.00      0.00      0.00         2
        2937       0.00      0.00      0.00        20
        2939       0.00      0.00      0.00         1
        2940       0.00      0.00      0.00         1
        2941       0.00      0.00      0.00         1
        2943       0.00      0.00      0.00         2
        2944       0.00      0.00      0.00         2
        2945       0.00      0.00      0.00         1
        2946       0.00      0.00      0.00         2
        2947       0.00      0.00      0.00         1
        2949       0.00      0.00      0.00         5
        2950       0.00      0.00      0.00         4
        2952       0.00      0.00      0.00         9
        2953       0.00      0.00      0.00         1
        2954       0.00      0.00      0.00         2
        2958       0.00      0.00      0.00         1
        2959       0.00      0.00      0.00         1
        2961       0.00      0.00      0.00         3
        2962       0.00      0.00      0.00         1
        2964       0.00      0.00      0.00         1
        2965       0.00      0.00      0.00         3
        2966       0.00      0.00      0.00         2
        2967       0.00      0.00      0.00         1
        2968       0.00      0.00      0.00         1
        2969       0.00      0.00      0.00         1
        2976       0.00      0.00      0.00         1
        2977       0.00      0.00      0.00         1
        2978       0.00      0.00      0.00         2
        2979       0.00      0.00      0.00         1
        2981       0.00      0.00      0.00         1
        2982       0.00      0.00      0.00         1
        2983       0.00      0.00      0.00         1
        2984       0.00      0.00      0.00         2
        2985       0.00      0.00      0.00         1
        2987       0.00      0.00      0.00         2
        2990       0.00      0.00      0.00         3
        2991       0.00      0.00      0.00         1
        2993       0.00      0.00      0.00         1
        2994       0.00      0.00      0.00         1
        2996       0.00      0.00      0.00         1
        2999       0.00      0.00      0.00         1
        3001       0.00      0.00      0.00         1
        3003       0.00      0.00      0.00         3
        3005       0.00      0.00      0.00         1
        3006       0.00      0.00      0.00         1
        3007       0.00      0.00      0.00         3
        3009       0.00      0.00      0.00         1
        3012       0.00      0.00      0.00         3
        3014       0.00      0.00      0.00         1
        3020       0.00      0.00      0.00         1
        3021       0.00      0.00      0.00         1
        3022       0.00      0.00      0.00         1
        3023       0.00      0.00      0.00         2
        3024       0.00      0.00      0.00         2
        3025       0.00      0.00      0.00         5
        3026       0.00      0.00      0.00         4
        3027       0.00      0.00      0.00         1
        3028       0.00      0.00      0.00         3
        3029       0.00      0.00      0.00         4
        3032       0.00      0.00      0.00         1
        3033       0.00      0.00      0.00         3
        3034       0.00      0.00      0.00         1
        3035       0.00      0.00      0.00         1
        3037       0.00      0.00      0.00         2
        3038       0.00      0.00      0.00         1
        3042       0.00      0.00      0.00         4
        3043       0.00      0.00      0.00         3
        3044       0.00      0.00      0.00         1
        3045       0.00      0.00      0.00         1
        3047       0.00      0.00      0.00         3
        3048       0.00      0.00      0.00         1
        3056       0.00      0.00      0.00         1
        3057       0.00      0.00      0.00         1
        3059       0.00      0.00      0.00         3
        3061       0.00      0.00      0.00         1
        3063       0.00      0.00      0.00         1
        3064       0.00      0.00      0.00         1
        3065       0.00      0.00      0.00         4
        3068       0.00      0.00      0.00         6
        3070       0.00      0.00      0.00         3
        3071       0.00      0.00      0.00         2
        3072       0.00      0.00      0.00         4
        3073       0.00      0.00      0.00         5
        3074       0.00      0.00      0.00         6
        3075       0.00      0.00      0.00         4
        3078       0.00      0.00      0.00         1
        3079       0.00      0.00      0.00         2
        3080       0.00      0.00      0.00         1
        3081       0.00      0.00      0.00         1
        3084       0.00      0.00      0.00         1
        3087       0.00      0.00      0.00         2
        3088       0.00      0.00      0.00         1
        3090       0.00      0.00      0.00         2
        3091       0.00      0.00      0.00         9
        3098       0.00      0.00      0.00         3
        3099       0.00      0.00      0.00         5
        3100       0.00      0.00      0.00         1
        3101       0.00      0.00      0.00         2
        3102       0.00      0.00      0.00         2
        3103       0.00      0.00      0.00        10
        3104       0.00      0.00      0.00         1
        3105       0.00      0.00      0.00         7
        3108       0.00      0.00      0.00         4
        3110       0.00      0.00      0.00         1
        3111       0.00      0.00      0.00         1
        3114       0.00      0.00      0.00         2
        3115       0.00      0.00      0.00         1
        3117       0.00      0.00      0.00         1
        3118       0.00      0.00      0.00         2
        3120       0.00      0.00      0.00         2
        3124       0.00      0.00      0.00         2
        3125       0.00      0.00      0.00         1
        3128       0.00      0.00      0.00         1
        3130       0.00      0.00      0.00         1
        3132       0.00      0.00      0.00         1
        3133       0.00      0.00      0.00         1
        3134       0.00      0.00      0.00         1
        3135       0.00      0.00      0.00         2
        3136       0.00      0.00      0.00         1
        3139       0.00      0.00      0.00         4
        3141       0.00      0.00      0.00         4
        3142       0.00      0.00      0.00         1
        3144       0.00      0.00      0.00         3
        3145       0.00      0.00      0.00         1
        3149       0.00      0.00      0.00         1
        3150       0.00      0.00      0.00         3
        3152       0.00      0.00      0.00         2
        3155       0.00      0.00      0.00         1
        3160       0.00      0.00      0.00         1
        3161       0.00      0.00      0.00         2
        3164       0.00      0.00      0.00         2
        3165       0.00      0.00      0.00         1
        3166       0.00      0.00      0.00         1
        3168       0.00      0.00      0.00         1
        3169       0.00      0.00      0.00         3
        3172       0.00      0.00      0.00         3
        3174       0.00      0.00      0.00         2
        3175       0.00      0.00      0.00         1
        3177       0.00      0.00      0.00         1
        3179       0.00      0.00      0.00         1
        3180       0.00      0.00      0.00         1
        3181       0.00      0.00      0.00         1
        3182       0.00      0.00      0.00         2
        3183       0.00      0.00      0.00         1
        3184       0.00      0.00      0.00         1
        3185       0.00      0.00      0.00         1
        3186       0.00      0.00      0.00         1
        3188       0.00      0.00      0.00         1
        3189       0.00      0.00      0.00         1
        3192       0.00      0.00      0.00         1
        3193       0.00      0.00      0.00         2
        3194       0.00      0.00      0.00         1
        3196       0.00      0.00      0.00         2
        3199       0.00      0.00      0.00         1
        3203       0.00      0.00      0.00         1
        3204       0.00      0.00      0.00         2
        3206       0.00      0.00      0.00         1
        3207       0.00      0.00      0.00         2
        3208       0.00      0.00      0.00         1
        3209       0.00      0.00      0.00         2
        3211       0.00      0.00      0.00         2
        3212       0.00      0.00      0.00         1
        3214       0.00      0.00      0.00         4
        3215       0.00      0.00      0.00         6
        3216       0.00      0.00      0.00         1
        3219       0.00      0.00      0.00         3
        3220       0.00      0.00      0.00         1
        3221       0.00      0.00      0.00         4
        3222       0.00      0.00      0.00         1
        3224       0.00      0.00      0.00         2
        3225       0.00      0.00      0.00         2
        3226       0.00      0.00      0.00         3
        3227       0.00      0.00      0.00         3
        3228       0.00      0.00      0.00         1
        3230       0.00      0.00      0.00         2
        3231       0.00      0.00      0.00         2
        3232       0.00      0.00      0.00         2
        3235       0.00      0.00      0.00         1
        3237       0.00      0.00      0.00         3
        3238       0.00      0.00      0.00         4
        3239       0.00      0.00      0.00         2
        3241       0.00      0.00      0.00         1
        3242       0.00      0.00      0.00         1
        3244       0.00      0.00      0.00         2
        3245       0.00      0.00      0.00         2
        3246       0.00      0.00      0.00         1
        3251       0.00      0.00      0.00         1
        3252       0.00      0.00      0.00         2
        3253       0.00      0.00      0.00         2
        3254       0.00      0.00      0.00         1
        3257       0.00      0.00      0.00         2
        3259       0.00      0.00      0.00         1
        3261       0.00      0.00      0.00         1
        3262       0.00      0.00      0.00         1
        3264       0.00      0.00      0.00         1
        3266       0.00      0.00      0.00         4
        3268       0.00      0.00      0.00         1
        3269       0.00      0.00      0.00         2
        3271       0.00      0.00      0.00         1
        3272       0.00      0.00      0.00         1
        3276       0.00      0.00      0.00         1
        3278       0.00      0.00      0.00         3
        3279       0.00      0.00      0.00         2
        3280       0.00      0.00      0.00         3
        3282       0.00      0.00      0.00         1
        3283       0.00      0.00      0.00         4
        3284       0.00      0.00      0.00         2
        3285       0.00      0.00      0.00         1
        3289       0.00      0.00      0.00         1
        3291       0.00      0.00      0.00         2
        3292       0.00      0.00      0.00         3
        3294       0.00      0.00      0.00         1
        3295       0.00      0.00      0.00         1
        3297       0.00      0.00      0.00         1
        3298       0.00      0.00      0.00         2
        3299       0.00      0.00      0.00         1
        3300       0.00      0.00      0.00         1
        3301       0.00      0.00      0.00         2
        3305       0.00      0.00      0.00         1
        3307       0.00      0.00      0.00         2
        3308       0.00      0.00      0.00         2
        3309       0.00      0.00      0.00         1
        3314       0.00      0.00      0.00         1
        3316       0.00      0.00      0.00         1
        3318       0.00      0.00      0.00         2
        3321       0.00      0.00      0.00        18
        3322       0.00      0.00      0.00         1
        3323       0.00      0.00      0.00         4
        3324       0.00      0.00      0.00         3
        3325       0.00      0.00      0.00         2
        3326       0.00      0.00      0.00         5
        3327       0.00      0.00      0.00         4
        3328       0.00      0.00      0.00         1
        3329       0.00      0.00      0.00         1
        3330       0.00      0.00      0.00         3
        3331       0.00      0.00      0.00         1
        3333       0.00      0.00      0.00         2
        3337       0.00      0.00      0.00         2
        3339       0.00      0.00      0.00         1
        3343       0.00      0.00      0.00         3
        3344       0.00      0.00      0.00         1
        3345       0.00      0.00      0.00         1
        3346       0.00      0.00      0.00         3
        3347       0.00      0.00      0.00         4
        3356       0.00      0.00      0.00         3
        3358       0.00      0.00      0.00         3
        3360       0.00      0.00      0.00         3
        3361       0.00      0.00      0.00         1
        3363       0.00      0.00      0.00         1
        3364       0.00      0.00      0.00         1
        3365       0.00      0.00      0.00         1
        3366       0.00      0.00      0.00         1
        3367       0.00      0.00      0.00         5
        3368       0.00      0.00      0.00         1
        3369       0.00      0.00      0.00         2
        3370       0.00      0.00      0.00         1
        3373       0.00      0.00      0.00         1
        3374       0.00      0.00      0.00         2
        3375       0.00      0.00      0.00         3
        3376       0.00      0.00      0.00         1
        3382       0.00      0.00      0.00         1
        3387       0.00      0.00      0.00         2
        3390       0.00      0.00      0.00         1
        3391       0.00      0.00      0.00         8
        3395       0.00      0.00      0.00         2
        3396       0.00      0.00      0.00         1
        3398       0.00      0.00      0.00         9
        3399       0.00      0.00      0.00        12
        3400       0.00      0.00      0.00         8
        3401       0.00      0.00      0.00         4
        3403       0.00      0.00      0.00         2
        3405       0.00      0.00      0.00         1
        3409       0.00      0.00      0.00         3
        3411       0.00      0.00      0.00         3
        3419       0.00      0.00      0.00         1
        3424       0.00      0.00      0.00         1
        3425       0.00      0.00      0.00        12
        3427       0.00      0.00      0.00         5
        3430       0.00      0.00      0.00         1
        3433       0.00      0.00      0.00         1
        3436       0.00      0.00      0.00         1
        3439       0.00      0.00      0.00         1
        3441       0.00      0.00      0.00         1
        3442       0.00      0.00      0.00         1
        3444       0.00      0.00      0.00         1
        3445       0.00      0.00      0.00         4
        3446       0.00      0.00      0.00         2
        3448       0.00      0.00      0.00         1
        3449       0.00      0.00      0.00         1
        3450       0.00      0.00      0.00         2
        3451       0.00      0.00      0.00         1
        3452       0.00      0.00      0.00         1
        3453       0.00      0.00      0.00         5
        3454       0.00      0.00      0.00         1
        3455       0.00      0.00      0.00         1
        3457       0.00      0.00      0.00         5
        3459       0.00      0.00      0.00         1
        3460       0.00      0.00      0.00         1
        3463       0.00      0.00      0.00         3
        3465       0.00      0.00      0.00         5
        3466       0.00      0.00      0.00         3
        3467       0.00      0.00      0.00         3
        3468       0.00      0.00      0.00         6
        3469       0.00      0.00      0.00         2
        3470       0.00      0.00      0.00         8
        3473       0.00      0.00      0.00         1
        3475       0.00      0.00      0.00         1
        3484       0.00      0.00      0.00         2
        3485       0.00      0.00      0.00         6
        3486       0.00      0.00      0.00         1
        3487       0.00      0.00      0.00         2
        3488       0.00      0.00      0.00         2
        3489       0.00      0.00      0.00         3
        3490       0.00      0.00      0.00         4
        3491       0.00      0.00      0.00         1
        3493       0.00      0.00      0.00         2
        3495       0.00      0.00      0.00         2
        3496       0.00      0.00      0.00         2
        3497       0.00      0.00      0.00         1
        3498       0.00      0.00      0.00         2
        3499       0.00      0.00      0.00         2
        3500       0.00      0.00      0.00         2
        3503       0.00      0.00      0.00         1
        3506       0.00      0.00      0.00         2
        3511       0.00      0.00      0.00         1
        3513       0.00      0.00      0.00         1
        3515       0.00      0.00      0.00         1
        3516       0.00      0.00      0.00         1
        3519       0.00      0.00      0.00         1
        3521       0.00      0.00      0.00         1
        3524       0.00      0.00      0.00         1
        3527       0.00      0.00      0.00         1
        3529       0.00      0.00      0.00         1
        3531       0.00      0.00      0.00         1
        3533       0.00      0.00      0.00         1
        3536       0.00      0.00      0.00         1
        3537       0.00      0.00      0.00         2
        3538       0.00      0.00      0.00         1
        3541       0.00      0.00      0.00         1
        3542       0.00      0.00      0.00         1
        3543       0.00      0.00      0.00         1
        3544       0.00      0.00      0.00         3
        3546       0.00      0.00      0.00         1
        3548       0.00      0.00      0.00         2
        3551       0.00      0.00      0.00         2
        3554       0.00      0.00      0.00         1
        3555       0.00      0.00      0.00         2
        3563       0.00      0.00      0.00         1
        3564       0.00      0.00      0.00         1
        3565       0.00      0.00      0.00         3
        3566       0.00      0.00      0.00         1
        3567       0.00      0.00      0.00         1
        3570       0.00      0.00      0.00         1
        3573       0.00      0.00      0.00         1
        3577       0.00      0.00      0.00         1
        3578       0.00      0.00      0.00         2
        3588       0.00      0.00      0.00         1
        3589       0.00      0.00      0.00         2
        3595       0.00      0.00      0.00         1
        3599       0.00      0.00      0.00         4
        3602       0.00      0.00      0.00         1
        3603       0.00      0.00      0.00         1
        3604       0.00      0.00      0.00         1
        3605       0.00      0.00      0.00         1
        3606       0.00      0.00      0.00         2
        3612       0.00      0.00      0.00         3
        3614       0.00      0.00      0.00         2
        3616       0.00      0.00      0.00         2
        3617       0.00      0.00      0.00         2
        3618       0.00      0.00      0.00         2
        3619       0.00      0.00      0.00         1
        3626       0.00      0.00      0.00         1
        3628       0.00      0.00      0.00         1
        3630       0.00      0.00      0.00         1
        3631       0.00      0.00      0.00         1
        3632       0.00      0.00      0.00         2
        3633       0.00      0.00      0.00         1
        3635       0.00      0.00      0.00        65
        3636       0.00      0.00      0.00         1
        3637       0.00      0.00      0.00         1
        3639       0.00      0.00      0.00         1
        3641       0.00      0.00      0.00         1
        3643       0.00      0.00      0.00         3
        3644       0.00      0.00      0.00         1
        3645       0.00      0.00      0.00         1
        3650       0.00      0.00      0.00         2
        3652       0.00      0.00      0.00         1
        3654       0.00      0.00      0.00         2
        3656       0.00      0.00      0.00         1
        3659       0.00      0.00      0.00         2
        3668       0.00      0.00      0.00         1
        3669       0.00      0.00      0.00         2
        3674       0.00      0.00      0.00         1
        3675       0.00      0.00      0.00         3
        3676       0.00      0.00      0.00         1
        3678       0.00      0.00      0.00         2
        3683       0.00      0.00      0.00         3
        3690       0.00      0.00      0.00         2
        3698       0.00      0.00      0.00         1
        3699       0.00      0.00      0.00         1
        3701       0.00      0.00      0.00         3
        3702       0.00      0.00      0.00         1
        3704       0.00      0.00      0.00         1
        3706       0.00      0.00      0.00         2
        3712       0.00      0.00      0.00         4
        3713       0.00      0.00      0.00         3
        3714       0.00      0.00      0.00         3
        3716       0.00      0.00      0.00         2
        3718       0.00      0.00      0.00         1
        3724       0.00      0.00      0.00         1
        3726       0.00      0.00      0.00         2
        3731       0.00      0.00      0.00         1
        3732       0.00      0.00      0.00         1
        3733       0.00      0.00      0.00         1
        3735       0.00      0.00      0.00         1
        3738       0.00      0.00      0.00         1
        3741       0.00      0.00      0.00         1
        3743       0.00      0.00      0.00         1
        3747       0.00      0.00      0.00         2
        3751       0.00      0.00      0.00         2
        3757       0.00      0.00      0.00         1
        3758       0.00      0.00      0.00         2
        3762       0.00      0.00      0.00         2
        3768       0.00      0.00      0.00         1
        3772       0.00      0.00      0.00         1
        3773       0.00      0.00      0.00         2
        3776       0.00      0.00      0.00         1
        3779       0.00      0.00      0.00         1
        3780       0.00      0.00      0.00         2
        3781       0.00      0.00      0.00         1
        3782       0.00      0.00      0.00         2
        3785       0.00      0.00      0.00         1
        3788       0.00      0.00      0.00         1
        3789       0.00      0.00      0.00         5
        3792       0.00      0.00      0.00         2
        3795       0.00      0.00      0.00         2
        3796       0.00      0.00      0.00         1
        3797       0.00      0.00      0.00         2
        3798       0.00      0.00      0.00         1
        3800       0.00      0.00      0.00         1
        3804       0.00      0.00      0.00         4
        3805       0.00      0.00      0.00         4
        3806       0.00      0.00      0.00         3
        3811       0.00      0.00      0.00         3
        3812       0.00      0.00      0.00         3
        3813       0.00      0.00      0.00         1
        3814       0.00      0.00      0.00         2
        3815       0.00      0.00      0.00         1
        3816       0.00      0.00      0.00         1
        3819       0.00      0.00      0.00         1
        3823       0.00      0.00      0.00         6
        3824       0.00      0.00      0.00         1
        3827       0.00      0.00      0.00         1
        3828       0.00      0.00      0.00         4
        3832       0.00      0.00      0.00         3
        3834       0.00      0.00      0.00         2
        3835       0.00      0.00      0.00         4
        3838       0.00      0.00      0.00         1
        3841       0.00      0.00      0.00         1
        3843       0.00      0.00      0.00         1
        3844       0.00      0.00      0.00         2
        3845       0.00      0.00      0.00         1
        3847       0.00      0.00      0.00         1
        3850       0.00      0.00      0.00         1
        3853       0.00      0.00      0.00         1
        3855       0.00      0.00      0.00         1
        3856       0.00      0.00      0.00         2
        3858       0.00      0.00      0.00         3
        3860       0.00      0.00      0.00         1
        3866       0.00      0.00      0.00         1
        3869       0.00      0.00      0.00         2
        3878       0.00      0.00      0.00         1
        3886       0.00      0.00      0.00         3
        3887       0.00      0.00      0.00         3
        3891       0.00      0.00      0.00         1
        3893       0.00      0.00      0.00         4
        3894       0.00      0.00      0.00         2
        3898       0.00      0.00      0.00         2
        3907       0.00      0.00      0.00         1
        3911       0.00      0.00      0.00         1
        3912       0.00      0.00      0.00         2
        3916       0.00      0.00      0.00         1
        3917       0.00      0.00      0.00         1
        3927       0.00      0.00      0.00         1
        3931       0.00      0.00      0.00         2
        3937       0.00      0.00      0.00         1
        3938       0.00      0.00      0.00         2
        3940       0.00      0.00      0.00         1
        3942       0.00      0.00      0.00         3
        3945       0.00      0.00      0.00         1
        3947       0.00      0.00      0.00         2
        3948       0.00      0.00      0.00         2
        3949       0.00      0.00      0.00         1
        3950       0.00      0.00      0.00         4
        3951       0.00      0.00      0.00         3
        3952       0.00      0.00      0.00         1
        3953       0.00      0.00      0.00         3
        3955       0.00      0.00      0.00         5
        3956       0.00      0.00      0.00         3
        3957       0.00      0.00      0.00         1
        3960       0.00      0.00      0.00         3
        3961       0.00      0.00      0.00         2
        3966       0.00      0.00      0.00         1
        3968       0.00      0.00      0.00         1
        3969       0.00      0.00      0.00         2
        3970       0.00      0.00      0.00         1
        3972       0.00      0.00      0.00         1
        3973       0.00      0.00      0.00         1
        3974       0.00      0.00      0.00         1
        3976       0.00      0.00      0.00         1
        3977       0.00      0.00      0.00         1
        3981       0.00      0.00      0.00         1
        3982       0.00      0.00      0.00         2
        3985       0.00      0.00      0.00         1
        3990       0.00      0.00      0.00         1
        3992       0.00      0.00      0.00         2
        3993       0.00      0.00      0.00         2
        3996       0.00      0.00      0.00         2
        3997       0.00      0.00      0.00         1
        3998       0.00      0.00      0.00         1
        3999       0.00      0.00      0.00         1
        4002       0.00      0.00      0.00         1
        4005       0.00      0.00      0.00         1
        4007       0.00      0.00      0.00         1
        4012       0.00      0.00      0.00         2
        4015       0.00      0.00      0.00         1
        4016       0.00      0.00      0.00         1
        4018       0.00      0.00      0.00         1
        4019       0.00      0.00      0.00         1
        4022       0.00      0.00      0.00         1
        4023       0.00      0.00      0.00         1
        4024       0.00      0.00      0.00         2
        4031       0.00      0.00      0.00         1
        4033       0.00      0.00      0.00         3
        4034       0.00      0.00      0.00         1
        4035       0.00      0.00      0.00         1
        4036       0.00      0.00      0.00         2
        4037       0.00      0.00      0.00         1
        4040       0.00      0.00      0.00         1
        4041       0.00      0.00      0.00         1
        4042       0.00      0.00      0.00         2
        4048       0.00      0.00      0.00         1
        4052       0.00      0.00      0.00         2
        4055       0.00      0.00      0.00         1
        4062       0.00      0.00      0.00         1
        4066       0.00      0.00      0.00         1
        4074       0.00      0.00      0.00         1
        4076       0.00      0.00      0.00         3
        4077       0.00      0.00      0.00         2
        4078       0.00      0.00      0.00         1
        4080       0.00      0.00      0.00         2
        4081       0.00      0.00      0.00         2
        4085       0.00      0.00      0.00         8
        4087       0.00      0.00      0.00         1
        4088       0.00      0.00      0.00         1
        4089       0.00      0.00      0.00         1
        4091       0.00      0.00      0.00         1
        4092       0.00      0.00      0.00         1
        4095       0.00      0.00      0.00         4
        4098       0.00      0.00      0.00         2
        4100       0.00      0.00      0.00         3
        4103       0.00      0.00      0.00         4
        4104       0.00      0.00      0.00         2
        4105       0.00      0.00      0.00         4
        4107       0.00      0.00      0.00         4
        4108       0.00      0.00      0.00         2
        4109       0.00      0.00      0.00         2
        4111       0.00      0.00      0.00         2
        4112       0.00      0.00      0.00         2
        4113       0.00      0.00      0.00         1
        4115       0.00      0.00      0.00         2
        4116       0.00      0.00      0.00         1
        4118       0.00      0.00      0.00         3
        4120       0.00      0.00      0.00         1
        4121       0.00      0.00      0.00         1
        4122       0.00      0.00      0.00         3
        4125       0.00      0.00      0.00         2
        4126       0.00      0.00      0.00         2
        4128       0.00      0.00      0.00         5
        4129       0.00      0.00      0.00         1
        4133       0.00      0.00      0.00         1
        4134       0.00      0.00      0.00         1
        4136       0.00      0.00      0.00         1
        4137       0.00      0.00      0.00         2
        4139       0.00      0.00      0.00         1
        4143       0.00      0.00      0.00         1
        4145       0.00      0.00      0.00         6
        4147       0.00      0.00      0.00         2
        4148       0.00      0.00      0.00         1
        4149       0.00      0.00      0.00         5
        4151       0.00      0.00      0.00         6
        4152       0.00      0.00      0.00         2
        4153       0.00      0.00      0.00         2
        4156       0.00      0.00      0.00         2
        4160       0.00      0.00      0.00         1
        4164       0.00      0.00      0.00         5
        4165       0.00      0.00      0.00         1
        4166       0.00      0.00      0.00         2
        4167       0.00      0.00      0.00         3
        4168       0.00      0.00      0.00         2
        4176       0.00      0.00      0.00         2
        4177       0.00      0.00      0.00         6
        4180       0.00      0.00      0.00         1
        4182       0.00      0.00      0.00         1
        4183       0.00      0.00      0.00         2
        4185       0.00      0.00      0.00         1
        4189       0.00      0.00      0.00         1
        4191       0.00      0.00      0.00         1
        4193       0.00      0.00      0.00         2
        4194       0.00      0.00      0.00         2
        4197       0.00      0.00      0.00         3
        4203       0.00      0.00      0.00         2
        4204       0.00      0.00      0.00         1
        4207       0.00      0.00      0.00         1
        4208       0.00      0.00      0.00         2
        4210       0.00      0.00      0.00         1
        4212       0.00      0.00      0.00        10
        4213       0.00      0.00      0.00         8
        4224       0.00      0.00      0.00         2
        4225       0.00      0.00      0.00         2
        4232       0.00      0.00      0.00         1
        4233       0.00      0.00      0.00         1
        4236       0.00      0.00      0.00         2
        4238       0.00      0.00      0.00         2
        4242       0.00      0.00      0.00         1
        4244       0.00      0.00      0.00         1
        4245       0.00      0.00      0.00         2
        4255       0.00      0.00      0.00         2
        4257       0.00      0.00      0.00         1
        4258       0.00      0.00      0.00         2
        4259       0.00      0.00      0.00         3
        4260       0.00      0.00      0.00         4
        4266       0.00      0.00      0.00         1
        4267       0.00      0.00      0.00         1
        4271       0.00      0.00      0.00         1
        4277       0.00      0.00      0.00         1
        4278       0.00      0.00      0.00         2
        4280       0.00      0.00      0.00         1
        4283       0.00      0.00      0.00         1
        4289       0.00      0.00      0.00         2
        4294       0.00      0.00      0.00         2
        4296       0.00      0.00      0.00         3
        4297       0.00      0.00      0.00         4
        4304       0.00      0.00      0.00         1
        4307       0.00      0.00      0.00         1
        4309       0.00      0.00      0.00         1
        4310       0.00      0.00      0.00         1
        4312       0.00      0.00      0.00         1
        4314       0.00      0.00      0.00         1
        4316       0.00      0.00      0.00         2
        4317       0.00      0.00      0.00         1
        4318       0.00      0.00      0.00         1
        4323       0.00      0.00      0.00         1
        4328       0.00      0.00      0.00         4
        4329       0.00      0.00      0.00         1
        4330       0.00      0.00      0.00         1
        4335       0.00      0.00      0.00         1
        4336       0.00      0.00      0.00         2
        4338       0.00      0.00      0.00         2
        4339       0.00      0.00      0.00         2
        4340       0.00      0.00      0.00         1
        4341       0.00      0.00      0.00         1
        4343       0.00      0.00      0.00         1
        4344       0.00      0.00      0.00         1
        4345       0.00      0.00      0.00         2
        4347       0.00      0.00      0.00         1
        4357       0.00      0.00      0.00         1
        4358       0.00      0.00      0.00         2
        4359       0.00      0.00      0.00         3
        4360       0.00      0.00      0.00         1
        4365       0.00      0.00      0.00         2
        4371       0.00      0.00      0.00         1
        4372       0.00      0.00      0.00         2
        4373       0.00      0.00      0.00         1
        4374       0.00      0.00      0.00         5
        4377       0.00      0.00      0.00         1
        4378       0.00      0.00      0.00         2
        4380       0.00      0.00      0.00         1
        4382       0.00      0.00      0.00         1
        4388       0.00      0.00      0.00         2
        4391       0.00      0.00      0.00         1
        4394       0.00      0.00      0.00         3
        4395       0.00      0.00      0.00         2
        4396       0.00      0.00      0.00         2
        4400       0.00      0.00      0.00         1
        4401       0.00      0.00      0.00         1
        4402       0.00      0.00      0.00         3
        4403       0.00      0.00      0.00         1
        4404       0.00      0.00      0.00         5
        4405       0.00      0.00      0.00         2
        4406       0.00      0.00      0.00         1
        4411       0.00      0.00      0.00         2
        4415       0.00      0.00      0.00         2
        4424       0.00      0.00      0.00         1
        4426       0.00      0.00      0.00         2
        4429       0.00      0.00      0.00         2
        4430       0.00      0.00      0.00         1
        4434       0.00      0.00      0.00         1
        4438       0.00      0.00      0.00         2
        4439       0.00      0.00      0.00         7
        4441       0.00      0.00      0.00         1
        4446       0.00      0.00      0.00         1
        4447       0.00      0.00      0.00         1
        4449       0.00      0.00      0.00         1
        4453       0.00      0.00      0.00         1
        4454       0.00      0.00      0.00         2
        4455       0.00      0.00      0.00         5
        4461       0.00      0.00      0.00         1
        4463       0.00      0.00      0.00         1
        4464       0.00      0.00      0.00         1
        4466       0.00      0.00      0.00         1
        4467       0.00      0.00      0.00         2
        4468       0.00      0.00      0.00         2
        4476       0.00      0.00      0.00         3
        4477       0.00      0.00      0.00         1
        4479       0.00      0.00      0.00         1
        4483       0.00      0.00      0.00         1
        4487       0.00      0.00      0.00         6
        4489       0.00      0.00      0.00         5
        4491       0.00      0.00      0.00         2
        4493       0.00      0.00      0.00         1
        4494       0.00      0.00      0.00         1
        4496       0.00      0.00      0.00         3
        4498       0.00      0.00      0.00         1
        4501       0.00      0.00      0.00         1
        4502       0.00      0.00      0.00         1
        4506       0.00      0.00      0.00         1
        4509       0.00      0.00      0.00         1
        4511       0.00      0.00      0.00         1
        4515       0.00      0.00      0.00         2
        4519       0.00      0.00      0.00         1
        4528       0.00      0.00      0.00         1
        4529       0.00      0.00      0.00         2
        4531       0.00      0.00      0.00         2
        4539       0.00      0.00      0.00         1
        4540       0.00      0.00      0.00         2
        4541       0.00      0.00      0.00         1
        4543       0.00      0.00      0.00         2
        4544       0.00      0.00      0.00         1
        4550       0.00      0.00      0.00         1
        4555       0.00      0.00      0.00         5
        4561       0.00      0.00      0.00         2
        4563       0.00      0.00      0.00         2
        4564       0.00      0.00      0.00         1
        4566       0.00      0.00      0.00         3
        4571       0.00      0.00      0.00         1
        4581       0.00      0.00      0.00         1
        4584       0.00      0.00      0.00         1
        4588       0.00      0.00      0.00         1
        4589       0.00      0.00      0.00         3
        4591       0.00      0.00      0.00         2
        4593       0.00      0.00      0.00         2
        4594       0.00      0.00      0.00         2
        4598       0.00      0.00      0.00         2
        4599       0.00      0.00      0.00         2
        4602       0.00      0.00      0.00         1
        4603       0.00      0.00      0.00         1
        4604       0.00      0.00      0.00         1
        4605       0.00      0.00      0.00         2
        4606       0.00      0.00      0.00         1
        4608       0.00      0.00      0.00         1
        4611       0.00      0.00      0.00         1
        4613       0.00      0.00      0.00         4
        4614       0.00      0.00      0.00         5
        4616       0.00      0.00      0.00         3
        4618       0.00      0.00      0.00         2
        4619       0.00      0.00      0.00         1
        4620       0.00      0.00      0.00         2
        4623       0.00      0.00      0.00         3
        4625       0.00      0.00      0.00         1
        4626       0.00      0.00      0.00         3
        4628       0.00      0.00      0.00         1
        4629       0.00      0.00      0.00         4
        4631       0.00      0.00      0.00         1
        4632       0.00      0.00      0.00         1
        4637       0.00      0.00      0.00         4
        4651       0.00      0.00      0.00         1
        4653       0.00      0.00      0.00         1
        4655       0.00      0.00      0.00         1
        4657       0.00      0.00      0.00         4
        4658       0.00      0.00      0.00         2
        4660       0.00      0.00      0.00         1
        4666       0.00      0.00      0.00         1
        4677       0.00      0.00      0.00         3
        4680       0.00      0.00      0.00         1
        4681       0.00      0.00      0.00         1
        4682       0.00      0.00      0.00         1
        4685       0.00      0.00      0.00         1
        4689       0.00      0.00      0.00         1
        4695       0.00      0.00      0.00         2
        4696       0.00      0.00      0.00         1
        4698       0.00      0.00      0.00         2
        4700       0.00      0.00      0.00         2
        4705       0.00      0.00      0.00         2
        4710       0.00      0.00      0.00         1
        4715       0.00      0.00      0.00         2
        4721       0.00      0.00      0.00         2
        4722       0.00      0.00      0.00         1
        4723       0.00      0.00      0.00         1
        4731       0.00      0.00      0.00         2
        4732       0.00      0.00      0.00         3
        4735       0.00      0.00      0.00         1
        4737       0.00      0.00      0.00         1
        4740       0.00      0.00      0.00         1
        4741       0.00      0.00      0.00         3
        4748       0.00      0.00      0.00         1
        4753       0.00      0.00      0.00         3
        4755       0.00      0.00      0.00         1
        4760       0.00      0.00      0.00         1
        4761       0.00      0.00      0.00         1
        4762       0.00      0.00      0.00         1
        4764       0.00      0.00      0.00         2
        4765       0.00      0.00      0.00         1
        4771       0.00      0.00      0.00         1
        4772       0.00      0.00      0.00         1
        4775       0.00      0.00      0.00         1
        4776       0.00      0.00      0.00         2
        4779       0.00      0.00      0.00         1
        4782       0.00      0.00      0.00         1
        4785       0.00      0.00      0.00         1
        4788       0.00      0.00      0.00         1
        4792       0.00      0.00      0.00         2
        4796       0.00      0.00      0.00         2
        4802       0.00      0.00      0.00         1
        4803       0.00      0.00      0.00         1
        4808       0.00      0.00      0.00         1
        4822       0.00      0.00      0.00         2
        4825       0.00      0.00      0.00         2
        4837       0.00      0.00      0.00         2
        4839       0.00      0.00      0.00         1
        4847       0.00      0.00      0.00         1
        4848       0.00      0.00      0.00         3
        4850       0.00      0.00      0.00         1
        4851       0.00      0.00      0.00         1
        4859       0.00      0.00      0.00         1
        4862       0.00      0.00      0.00         1
        4863       0.00      0.00      0.00         1
        4868       0.00      0.00      0.00         1
        4872       0.00      0.00      0.00         1
        4874       0.00      0.00      0.00         2
        4878       0.00      0.00      0.00         3
        4879       0.00      0.00      0.00         2
        4881       0.00      0.00      0.00         1
        4883       0.00      0.00      0.00         1
        4884       0.00      0.00      0.00         5
        4889       0.00      0.00      0.00         1
        4894       0.00      0.00      0.00         2
        4895       0.00      0.00      0.00         3
        4898       0.00      0.00      0.00         1
        4904       0.00      0.00      0.00         1
        4916       0.00      0.00      0.00         1
        4917       0.00      0.00      0.00         1
        4923       0.00      0.00      0.00         2
        4924       0.00      0.00      0.00         1
        4925       0.00      0.00      0.00         2
        4926       0.00      0.00      0.00         1
        4927       0.00      0.00      0.00         1
        4930       0.00      0.00      0.00         1
        4933       0.00      0.00      0.00         3
        4935       0.00      0.00      0.00         1
        4936       0.00      0.00      0.00         8
        4937       0.00      0.00      0.00         3
        4939       0.00      0.00      0.00         6
        4946       0.00      0.00      0.00         6
        4951       0.00      0.00      0.00         2
        4956       0.00      0.00      0.00         1
        4957       0.00      0.00      0.00         2
        4959       0.00      0.00      0.00         1
        4960       0.00      0.00      0.00         1
        4964       0.00      0.00      0.00         1
        4968       0.00      0.00      0.00         3
        4976       0.00      0.00      0.00         1
        4978       0.00      0.00      0.00         1
        4984       0.00      0.00      0.00         1
        4985       0.00      0.00      0.00         2
        4986       0.00      0.00      0.00         1
        4987       0.00      0.00      0.00         3
        4992       0.00      0.00      0.00         5
        4993       0.00      0.00      0.00         1
        4995       0.00      0.00      0.00         2
        5001       0.00      0.00      0.00         3
        5002       0.00      0.00      0.00         2
        5005       0.00      0.00      0.00         2
        5009       0.00      0.00      0.00         1
        5014       0.00      0.00      0.00         1
        5016       0.00      0.00      0.00         1
        5029       0.00      0.00      0.00         1
        5030       0.00      0.00      0.00         1
        5036       0.00      0.00      0.00         1
        5037       0.00      0.00      0.00         1
        5038       0.00      0.00      0.00         1
        5042       0.00      0.00      0.00         1
        5045       0.00      0.00      0.00         1
        5046       0.00      0.00      0.00         1
        5050       0.00      0.00      0.00         2
        5055       0.00      0.00      0.00         4
        5060       0.00      0.00      0.00         1
        5062       0.00      0.00      0.00         1
        5065       0.00      0.00      0.00         2
        5070       0.00      0.00      0.00         1
        5071       0.00      0.00      0.00         1
        5077       0.00      0.00      0.00         2
        5078       0.00      0.00      0.00         3
        5085       0.00      0.00      0.00         4
        5086       0.00      0.00      0.00         1
        5087       0.00      0.00      0.00         2
        5088       0.00      0.00      0.00         1
        5091       0.00      0.00      0.00         1
        5092       0.00      0.00      0.00         1
        5095       0.00      0.00      0.00         1
        5096       0.00      0.00      0.00         1
        5097       0.00      0.00      0.00         3
        5101       0.00      0.00      0.00         1
        5102       0.00      0.00      0.00         2
        5110       0.00      0.00      0.00         4
        5114       0.00      0.00      0.00         2
        5118       0.00      0.00      0.00         1
        5122       0.00      0.00      0.00         1
        5124       0.00      0.00      0.00         1
        5125       0.00      0.00      0.00         1
        5126       0.00      0.00      0.00        10
        5127       0.00      0.00      0.00         1
        5133       0.00      0.00      0.00         2
        5134       0.00      0.00      0.00         1
        5135       0.00      0.00      0.00         2
        5140       0.00      0.00      0.00         1
        5142       0.00      0.00      0.00         1
        5147       0.00      0.00      0.00         1
        5154       0.00      0.00      0.00         2
        5155       0.00      0.00      0.00         2
        5156       0.00      0.00      0.00         1
        5157       0.00      0.00      0.00         1
        5160       0.00      0.00      0.00         1
        5172       0.00      0.00      0.00         3
        5173       0.00      0.00      0.00         1
        5180       0.00      0.00      0.00         1
        5187       0.00      0.00      0.00         1
        5192       0.00      0.00      0.00         3
        5197       0.00      0.00      0.00         2
        5201       0.00      0.00      0.00         1
        5207       0.00      0.00      0.00         1
        5210       0.00      0.00      0.00         1
        5211       0.00      0.00      0.00         1
        5213       0.00      0.00      0.00         2
        5215       0.00      0.00      0.00         1
        5226       0.00      0.00      0.00         1
        5228       0.00      0.00      0.00         6
        5229       0.00      0.00      0.00         1
        5239       0.00      0.00      0.00         1
        5240       0.00      0.00      0.00         2
        5243       0.00      0.00      0.00         1
        5244       0.00      0.00      0.00         4
        5245       0.00      0.00      0.00         1
        5247       0.00      0.00      0.00         1
        5248       0.00      0.00      0.00         2
        5251       0.00      0.00      0.00         2
        5254       0.00      0.00      0.00         2
        5260       0.00      0.00      0.00         2
        5265       0.00      0.00      0.00         1
        5273       0.00      0.00      0.00         3
        5275       0.00      0.00      0.00         2
        5276       0.00      0.00      0.00         1
        5277       0.00      0.00      0.00         2
        5283       0.00      0.00      0.00         5
        5292       0.00      0.00      0.00         2
        5298       0.00      0.00      0.00         1
        5303       0.00      0.00      0.00         2
        5304       0.00      0.00      0.00         1
        5309       0.00      0.00      0.00         3
        5311       0.00      0.00      0.00         1
        5312       0.00      0.00      0.00         1
        5319       0.00      0.00      0.00         2
        5320       0.00      0.00      0.00         2
        5321       0.00      0.00      0.00         1
        5322       0.00      0.00      0.00         2
        5326       0.00      0.00      0.00         1
        5335       0.00      0.00      0.00         1
        5345       0.00      0.00      0.00         1
        5350       0.00      0.00      0.00         1
        5351       0.00      0.00      0.00         2
        5352       0.00      0.00      0.00         3
        5357       0.00      0.00      0.00         1
        5360       0.00      0.00      0.00         3
        5379       0.00      0.00      0.00         1
        5385       0.00      0.00      0.00         1
        5395       0.00      0.00      0.00         2
        5405       0.00      0.00      0.00         1
        5406       0.00      0.00      0.00         3
        5413       0.00      0.00      0.00         1
        5419       0.00      0.00      0.00         3
        5423       0.00      0.00      0.00         1
        5436       0.00      0.00      0.00         3
        5437       0.00      0.00      0.00         1
        5438       0.00      0.00      0.00         2
        5440       0.00      0.00      0.00         1
        5443       0.00      0.00      0.00         2
        5459       0.00      0.00      0.00         1
        5462       0.00      0.00      0.00         1
        5464       0.00      0.00      0.00         1
        5466       0.00      0.00      0.00         1
        5470       0.00      0.00      0.00         1
        5473       0.00      0.00      0.00         1
        5482       0.00      0.00      0.00         4
        5483       0.00      0.00      0.00         1
        5484       0.00      0.00      0.00         2
        5488       0.00      0.00      0.00         1
        5514       0.00      0.00      0.00         1
        5519       0.00      0.00      0.00         2
        5520       0.00      0.00      0.00         1
        5527       0.00      0.00      0.00         1
        5545       0.00      0.00      0.00         1
        5548       0.00      0.00      0.00         1
        5570       0.00      0.00      0.00         2
        5572       0.00      0.00      0.00         1
        5573       0.00      0.00      0.00         1
        5576       0.00      0.00      0.00         1
        5582       0.00      0.00      0.00         1
        5584       0.00      0.00      0.00         2
        5585       0.00      0.00      0.00         1
        5590       0.00      0.00      0.00         2
        5598       0.00      0.00      0.00         1
        5599       0.00      0.00      0.00         1
        5600       0.00      0.00      0.00         1
        5601       0.00      0.00      0.00         1
        5619       0.00      0.00      0.00         1
        5621       0.00      0.00      0.00         2
        5622       0.00      0.00      0.00         2
        5630       0.00      0.00      0.00         1
        5635       0.00      0.00      0.00         3
        5639       0.00      0.00      0.00         1
        5640       0.00      0.00      0.00         2
        5645       0.00      0.00      0.00         2
        5646       0.00      0.00      0.00         2
        5649       0.00      0.00      0.00         1
        5651       0.00      0.00      0.00         3
        5655       0.00      0.00      0.00         1
        5657       0.00      0.00      0.00         1
        5666       0.00      0.00      0.00         1
        5674       0.00      0.00      0.00         1
        5675       0.00      0.00      0.00         2
        5678       0.00      0.00      0.00         1
        5685       0.00      0.00      0.00         2
        5686       0.00      0.00      0.00         2
        5693       0.00      0.00      0.00         1
        5696       0.00      0.00      0.00         2
        5697       0.00      0.00      0.00         1
        5698       0.00      0.00      0.00         1
        5699       0.00      0.00      0.00         1
        5713       0.00      0.00      0.00         1
        5717       0.00      0.00      0.00         1
        5721       0.00      0.00      0.00         2
        5725       0.00      0.00      0.00         2
        5727       0.00      0.00      0.00         1
        5728       0.00      0.00      0.00         1
        5731       0.00      0.00      0.00         3
        5732       0.00      0.00      0.00         1
        5742       0.00      0.00      0.00         1
        5754       0.00      0.00      0.00         2
        5761       0.00      0.00      0.00         2
        5767       0.00      0.00      0.00         2
        5773       0.00      0.00      0.00         1
        5775       0.00      0.00      0.00         1
        5778       0.00      0.00      0.00         1
        5782       0.00      0.00      0.00         1
        5785       0.00      0.00      0.00         1
        5789       0.00      0.00      0.00         1
        5790       0.00      0.00      0.00         1
        5792       0.00      0.00      0.00         1
        5793       0.00      0.00      0.00         1
        5806       0.00      0.00      0.00         1
        5810       0.00      0.00      0.00         2
        5811       0.00      0.00      0.00         2
        5812       0.00      0.00      0.00         5
        5817       0.00      0.00      0.00         1
        5818       0.00      0.00      0.00         2
        5819       0.00      0.00      0.00         4
        5820       0.00      0.00      0.00         1
        5823       0.00      0.00      0.00         2
        5827       0.00      0.00      0.00         1
        5842       0.00      0.00      0.00         1
        5844       0.00      0.00      0.00         1
        5848       0.00      0.00      0.00         1
        5860       0.00      0.00      0.00         3
        5865       0.00      0.00      0.00         1
        5866       0.00      0.00      0.00         2
        5867       0.00      0.00      0.00         2
        5869       0.00      0.00      0.00         1
        5875       0.00      0.00      0.00         1
        5885       0.00      0.00      0.00         2
        5889       0.00      0.00      0.00         1
        5890       0.00      0.00      0.00         1
        5899       0.00      0.00      0.00         1
        5914       0.00      0.00      0.00         1
        5915       0.00      0.00      0.00         1
        5916       0.00      0.00      0.00         1
        5923       0.00      0.00      0.00         1
        5931       0.00      0.00      0.00         1
        5940       0.00      0.00      0.00         1
        5948       0.00      0.00      0.00         1
        5955       0.00      0.00      0.00         1
        5968       0.00      0.00      0.00         1
        5976       0.00      0.00      0.00         3
        5988       0.00      0.00      0.00         1
        5989       0.00      0.00      0.00         3
        5993       0.00      0.00      0.00         1
        5997       0.00      0.00      0.00         1
        6000       0.00      0.00      0.00         1
        6002       0.00      0.00      0.00         1
        6008       0.00      0.00      0.00         1
        6017       0.00      0.00      0.00         1
        6023       0.00      0.00      0.00         1
        6029       0.00      0.00      0.00         1
        6030       0.00      0.00      0.00         1
        6038       0.00      0.00      0.00         1
        6057       0.00      0.00      0.00         2
        6068       0.00      0.00      0.00         3
        6069       0.00      0.00      0.00         2
        6071       0.00      0.00      0.00         2
        6072       0.00      0.00      0.00         1
        6078       0.00      0.00      0.00         1
        6082       0.00      0.00      0.00         3
        6086       0.00      0.00      0.00         1
        6092       0.00      0.00      0.00         3
        6106       0.00      0.00      0.00        13
        6114       0.00      0.00      0.00         2
        6116       0.00      0.00      0.00         1
        6122       0.00      0.00      0.00         1
        6127       0.00      0.00      0.00         1
        6129       0.00      0.00      0.00         1
        6134       0.00      0.00      0.00         1
        6145       0.00      0.00      0.00         4
        6152       0.00      0.00      0.00         1
        6153       0.00      0.00      0.00         4
        6154       0.00      0.00      0.00         2
        6158       0.00      0.00      0.00         1
        6159       0.00      0.00      0.00         1
        6161       0.00      0.00      0.00         1
        6169       0.00      0.00      0.00         3
        6170       0.00      0.00      0.00         7
        6171       0.00      0.00      0.00         5
        6172       0.00      0.00      0.00         4
        6173       0.00      0.00      0.00         2
        6175       0.00      0.00      0.00         1
        6177       0.00      0.00      0.00         3
        6179       0.00      0.00      0.00         2
        6180       0.00      0.00      0.00         1
        6191       0.00      0.00      0.00         1
        6196       0.00      0.00      0.00         1
        6197       0.00      0.00      0.00         1
        6198       0.00      0.00      0.00         3
        6200       0.00      0.00      0.00         2
        6202       0.00      0.00      0.00         1
        6206       0.00      0.00      0.00         1
        6210       0.00      0.00      0.00         1
        6221       0.00      0.00      0.00         1
        6230       0.00      0.00      0.00         2
        6237       0.00      0.00      0.00         1
        6238       0.00      0.00      0.00         1
        6239       0.00      0.00      0.00         1
        6244       0.00      0.00      0.00         3
        6247       0.00      0.00      0.00         2
        6249       0.00      0.00      0.00         2
        6257       0.00      0.00      0.00         2
        6265       0.00      0.00      0.00         1
        6269       0.00      0.00      0.00         2
        6272       0.00      0.00      0.00         1
        6277       0.00      0.00      0.00         1
        6279       0.00      0.00      0.00         1
        6280       0.00      0.00      0.00         1
        6282       0.00      0.00      0.00         1
        6283       0.00      0.00      0.00         1
        6288       0.00      0.00      0.00         1
        6295       0.00      0.00      0.00         2
        6299       0.00      0.00      0.00         2
        6319       0.00      0.00      0.00         1
        6324       0.00      0.00      0.00         2
        6326       0.00      0.00      0.00         1
        6330       0.00      0.00      0.00         1
        6331       0.00      0.00      0.00         1
        6334       0.00      0.00      0.00         1
        6335       0.00      0.00      0.00         1
        6336       0.00      0.00      0.00         1
        6340       0.00      0.00      0.00         1
        6341       0.00      0.00      0.00         3
        6358       0.00      0.00      0.00         1
        6360       0.00      0.00      0.00         4
        6376       0.00      0.00      0.00         1
        6382       0.00      0.00      0.00         1
        6392       0.00      0.00      0.00         2
        6393       0.00      0.00      0.00         1
        6404       0.00      0.00      0.00         3
        6405       0.00      0.00      0.00         5
        6410       0.00      0.00      0.00         2
        6411       0.00      0.00      0.00         4
        6415       0.00      0.00      0.00         2
        6420       0.00      0.00      0.00         4
        6421       0.00      0.00      0.00         1
        6431       0.00      0.00      0.00         1
        6432       0.00      0.00      0.00         1
        6437       0.00      0.00      0.00         1
        6438       0.00      0.00      0.00         1
        6439       0.00      0.00      0.00         3
        6442       0.00      0.00      0.00         1
        6445       0.00      0.00      0.00        10
        6448       0.00      0.00      0.00         2
        6449       0.00      0.00      0.00         1
        6458       0.00      0.00      0.00         1
        6462       0.00      0.00      0.00         1
        6463       0.00      0.00      0.00         1
        6464       0.00      0.00      0.00         1
        6468       0.00      0.00      0.00         1
        6471       0.00      0.00      0.00         1
        6472       0.00      0.00      0.00         1
        6482       0.00      0.00      0.00         2
        6488       0.00      0.00      0.00         1
        6489       0.00      0.00      0.00         1
        6490       0.00      0.00      0.00         1
        6497       0.00      0.00      0.00         1
        6508       0.00      0.00      0.00         2
        6516       0.00      0.00      0.00         1
        6518       0.00      0.00      0.00         1
        6534       0.00      0.00      0.00         3
        6540       0.00      0.00      0.00         1
        6547       0.00      0.00      0.00         1
        6548       0.00      0.00      0.00         1
        6549       0.00      0.00      0.00         1
        6550       0.00      0.00      0.00         1
        6559       0.00      0.00      0.00         2
        6561       0.00      0.00      0.00         2
        6562       0.00      0.00      0.00         1
        6571       0.00      0.00      0.00         1
        6572       0.00      0.00      0.00         3
        6575       0.00      0.00      0.00         1
        6578       0.00      0.00      0.00         1
        6584       0.00      0.00      0.00         2
        6587       0.00      0.00      0.00         3
        6606       0.00      0.00      0.00         1
        6607       0.00      0.00      0.00         1
        6608       0.00      0.00      0.00         1
        6620       0.00      0.00      0.00         1
        6622       0.00      0.00      0.00         1
        6636       0.00      0.00      0.00         1
        6657       0.00      0.00      0.00         3
        6684       0.00      0.00      0.00         1
        6689       0.00      0.00      0.00         1
        6694       0.00      0.00      0.00         3
        6701       0.00      0.00      0.00         2
        6705       0.00      0.00      0.00         2
        6711       0.00      0.00      0.00         1
        6716       0.00      0.00      0.00         1
        6719       0.00      0.00      0.00         1
        6720       0.00      0.00      0.00         1
        6722       0.00      0.00      0.00         1
        6723       0.00      0.00      0.00         1
        6724       0.00      0.00      0.00         1
        6727       0.00      0.00      0.00         8
        6728       0.00      0.00      0.00         1
        6729       0.00      0.00      0.00         1
        6730       0.00      0.00      0.00         2
        6733       0.00      0.00      0.00         1
        6734       0.00      0.00      0.00         1
        6737       0.00      0.00      0.00         1
        6747       0.00      0.00      0.00         1
        6751       0.00      0.00      0.00         1
        6753       0.00      0.00      0.00         1
        6766       0.00      0.00      0.00         1
        6770       0.00      0.00      0.00         1
        6773       0.00      0.00      0.00         1
        6780       0.00      0.00      0.00         1
        6795       0.00      0.00      0.00         3
        6810       0.00      0.00      0.00         1
        6819       0.00      0.00      0.00         1
        6826       0.00      0.00      0.00         2
        6827       0.00      0.00      0.00         1
        6833       0.00      0.00      0.00         1
        6840       0.00      0.00      0.00         1
        6841       0.00      0.00      0.00         2
        6849       0.00      0.00      0.00         1
        6852       0.00      0.00      0.00         1
        6854       0.00      0.00      0.00         1
        6856       0.00      0.00      0.00         1
        6862       0.00      0.00      0.00         1
        6863       0.00      0.00      0.00         1
        6865       0.00      0.00      0.00         2
        6874       0.00      0.00      0.00         1
        6877       0.00      0.00      0.00         1
        6888       0.00      0.00      0.00         1
        6898       0.00      0.00      0.00         1
        6904       0.00      0.00      0.00         1
        6906       0.00      0.00      0.00         1
        6909       0.00      0.00      0.00         1
        6913       0.00      0.00      0.00         4
        6917       0.00      0.00      0.00         1
        6921       0.00      0.00      0.00         1
        6931       0.00      0.00      0.00         1
        6933       0.00      0.00      0.00         1
        6936       0.00      0.00      0.00         1
        6938       0.00      0.00      0.00         1
        6942       0.00      0.00      0.00         1
        6949       0.00      0.00      0.00         1
        6955       0.00      0.00      0.00         1
        6969       0.00      0.00      0.00         2
        6970       0.00      0.00      0.00         2
        6972       0.00      0.00      0.00         2
        6974       0.00      0.00      0.00         1
        6978       0.00      0.00      0.00         2
        6979       0.00      0.00      0.00         1
        6980       0.00      0.00      0.00         1
        6982       0.00      0.00      0.00         2
        6988       0.00      0.00      0.00         1
        6989       0.00      0.00      0.00         3
        6991       0.00      0.00      0.00         1
        6992       0.00      0.00      0.00         5
        6993       0.00      0.00      0.00         1
        6997       0.00      0.00      0.00         3
        7020       0.00      0.00      0.00         3
        7022       0.00      0.00      0.00         2
        7025       0.00      0.00      0.00         2
        7026       0.00      0.00      0.00         1
        7029       0.00      0.00      0.00         1
        7036       0.00      0.00      0.00         1
        7039       0.00      0.00      0.00         1
        7044       0.00      0.00      0.00         2
        7048       0.00      0.00      0.00         2
        7051       0.00      0.00      0.00         1
        7053       0.00      0.00      0.00         2
        7060       0.00      0.00      0.00         2
        7063       0.00      0.00      0.00         1
        7068       0.00      0.00      0.00         1
        7073       0.00      0.00      0.00         1
        7076       0.00      0.00      0.00         1
        7082       0.00      0.00      0.00         1
        7083       0.00      0.00      0.00         1
        7087       0.00      0.00      0.00         2
        7093       0.00      0.00      0.00         1
        7095       0.00      0.00      0.00         3
        7096       0.00      0.00      0.00         1
        7110       0.00      0.00      0.00         2
        7115       0.00      0.00      0.00         1
        7125       0.00      0.00      0.00         2
        7126       0.00      0.00      0.00         1
        7130       0.00      0.00      0.00         3
        7142       0.00      0.00      0.00         1
        7148       0.00      0.00      0.00         2
        7153       0.00      0.00      0.00         1
        7154       0.00      0.00      0.00         1
        7156       0.00      0.00      0.00         1
        7160       0.00      0.00      0.00         1
        7163       0.00      0.00      0.00         2
        7164       0.00      0.00      0.00         1
        7166       0.00      0.00      0.00         2
        7170       0.00      0.00      0.00         1
        7172       0.00      0.00      0.00         2
        7175       0.00      0.00      0.00         2
        7178       0.00      0.00      0.00         1
        7191       0.00      0.00      0.00         1
        7192       0.00      0.00      0.00         1
        7193       0.00      0.00      0.00         1
        7194       0.00      0.00      0.00         1
        7195       0.00      0.00      0.00         1
        7197       0.00      0.00      0.00         2
        7200       0.00      0.00      0.00         2
        7202       0.00      0.00      0.00         1
        7210       0.00      0.00      0.00         3
        7215       0.00      0.00      0.00         1
        7218       0.00      0.00      0.00         1
        7220       0.00      0.00      0.00         1
        7223       0.00      0.00      0.00         1
        7224       0.00      0.00      0.00         2
        7225       0.00      0.00      0.00         1
        7229       0.00      0.00      0.00         1
        7241       0.00      0.00      0.00         1
        7248       0.00      0.00      0.00         1
        7251       0.00      0.00      0.00         3
        7264       0.00      0.00      0.00         1
        7271       0.00      0.00      0.00         1
        7277       0.00      0.00      0.00         1
        7293       0.00      0.00      0.00         1
        7312       0.00      0.00      0.00         2
        7319       0.00      0.00      0.00         3
        7338       0.00      0.00      0.00         1
        7341       0.00      0.00      0.00         1
        7351       0.00      0.00      0.00         2
        7352       0.00      0.00      0.00         2
        7355       0.00      0.00      0.00         3
        7356       0.00      0.00      0.00         1
        7365       0.00      0.00      0.00         1
        7371       0.00      0.00      0.00         1
        7372       0.00      0.00      0.00         1
        7374       0.00      0.00      0.00         1
        7375       0.00      0.00      0.00         1
        7377       0.00      0.00      0.00         1
        7381       0.00      0.00      0.00         1
        7384       0.00      0.00      0.00         2
        7386       0.00      0.00      0.00         1
        7392       0.00      0.00      0.00         2
        7396       0.00      0.00      0.00         2
        7397       0.00      0.00      0.00         1
        7398       0.00      0.00      0.00         2
        7399       0.00      0.00      0.00         1
        7403       0.00      0.00      0.00         1
        7408       0.00      0.00      0.00         1
        7409       0.00      0.00      0.00         2
        7410       0.00      0.00      0.00         1
        7417       0.00      0.00      0.00         1
        7419       0.00      0.00      0.00         1
        7420       0.00      0.00      0.00         1
        7432       0.00      0.00      0.00         1
        7442       0.00      0.00      0.00         2
        7445       0.00      0.00      0.00         1
        7454       0.00      0.00      0.00         1
        7459       0.00      0.00      0.00         1
        7462       0.00      0.00      0.00         1
        7463       0.00      0.00      0.00         1
        7468       0.00      0.00      0.00         1
        7470       0.00      0.00      0.00         1
        7484       0.00      0.00      0.00         1
        7502       0.00      0.00      0.00         1
        7513       0.00      0.00      0.00         3
        7521       0.00      0.00      0.00         1
        7526       0.00      0.00      0.00         1
        7548       0.00      0.00      0.00         1
        7555       0.00      0.00      0.00         1
        7579       0.00      0.00      0.00         2
        7583       0.00      0.00      0.00         2
        7585       0.00      0.00      0.00         5
        7588       0.00      0.00      0.00         2
        7612       0.00      0.00      0.00         1
        7614       0.00      0.00      0.00         1
        7619       0.00      0.00      0.00         1
        7622       0.00      0.00      0.00         1
        7626       0.00      0.00      0.00         1
        7627       0.00      0.00      0.00         3
        7628       0.00      0.00      0.00         1
        7629       0.00      0.00      0.00         2
        7633       0.00      0.00      0.00         2
        7635       0.00      0.00      0.00         2
        7636       0.00      0.00      0.00         1
        7637       0.00      0.00      0.00         1
        7640       0.00      0.00      0.00         1
        7645       0.00      0.00      0.00         1
        7652       0.00      0.00      0.00         1
        7654       0.00      0.00      0.00         1
        7661       0.00      0.00      0.00         1
        7674       0.00      0.00      0.00         1
        7693       0.00      0.00      0.00         1
        7695       0.00      0.00      0.00         2
        7699       0.00      0.00      0.00         1
        7702       0.00      0.00      0.00         1
        7704       0.00      0.00      0.00         1
        7714       0.00      0.00      0.00         1
        7717       0.00      0.00      0.00         2
        7724       0.00      0.00      0.00         2
        7732       0.00      0.00      0.00         1
        7753       0.00      0.00      0.00         2
        7767       0.00      0.00      0.00         1
        7786       0.00      0.00      0.00         1
        7794       0.00      0.00      0.00         1
        7799       0.00      0.00      0.00         2
        7803       0.00      0.00      0.00         1
        7804       0.00      0.00      0.00         1
        7814       0.00      0.00      0.00         1
        7818       0.00      0.00      0.00         1
        7836       0.00      0.00      0.00         2
        7839       0.00      0.00      0.00         1
        7842       0.00      0.00      0.00         1
        7856       0.00      0.00      0.00         1
        7858       0.00      0.00      0.00         1
        7863       0.00      0.00      0.00         2
        7894       0.00      0.00      0.00         1
        7896       0.00      0.00      0.00         2
        7900       0.00      0.00      0.00         1
        7903       0.00      0.00      0.00         1
        7910       0.00      0.00      0.00         1
        7913       0.00      0.00      0.00         1
        7930       0.00      0.00      0.00         1
        7942       0.00      0.00      0.00         1
        7954       0.00      0.00      0.00         1
        7956       0.00      0.00      0.00         1
        7960       0.00      0.00      0.00         1
        7970       0.00      0.00      0.00         1
        7973       0.00      0.00      0.00         1
        7979       0.00      0.00      0.00         1
        7987       0.00      0.00      0.00         1
        7989       0.00      0.00      0.00         1
        8001       0.00      0.00      0.00         3
        8007       0.00      0.00      0.00         1
        8008       0.00      0.00      0.00         2
        8035       0.00      0.00      0.00         2
        8039       0.00      0.00      0.00         3
        8042       0.00      0.00      0.00         2
        8051       0.00      0.00      0.00         1
        8055       0.00      0.00      0.00         1
        8060       0.00      0.00      0.00         1
        8062       0.00      0.00      0.00         2
        8067       0.00      0.00      0.00         1
        8073       0.00      0.00      0.00         1
        8077       0.00      0.00      0.00         1
        8079       0.00      0.00      0.00         1
        8085       0.00      0.00      0.00         1
        8088       0.00      0.00      0.00         1
        8093       0.00      0.00      0.00         4
        8099       0.00      0.00      0.00         1
        8103       0.00      0.00      0.00         4
        8110       0.00      0.00      0.00         1
        8120       0.00      0.00      0.00         1
        8122       0.00      0.00      0.00         1
        8123       0.00      0.00      0.00         4
        8126       0.00      0.00      0.00         1
        8132       0.00      0.00      0.00         1
        8145       0.00      0.00      0.00         1
        8146       0.00      0.00      0.00         1
        8147       0.00      0.00      0.00         1
        8148       0.00      0.00      0.00         2
        8150       0.00      0.00      0.00         1
        8153       0.00      0.00      0.00         1
        8156       0.00      0.00      0.00         4
        8158       0.00      0.00      0.00         2
        8159       0.00      0.00      0.00         1
        8165       0.00      0.00      0.00         1
        8174       0.00      0.00      0.00         1
        8176       0.00      0.00      0.00         1
        8182       0.00      0.00      0.00         1
        8188       0.00      0.00      0.00         3
        8189       0.00      0.00      0.00         1
        8191       0.00      0.00      0.00         1
        8194       0.00      0.00      0.00         1
        8196       0.00      0.00      0.00         1
        8200       0.00      0.00      0.00         1
        8202       0.00      0.00      0.00         3
        8203       0.00      0.00      0.00         1
        8204       0.00      0.00      0.00         1
        8205       0.00      0.00      0.00         1
        8206       0.00      0.00      0.00         4
        8217       0.00      0.00      0.00         1
        8221       0.00      0.00      0.00         1
        8233       0.00      0.00      0.00         1
        8236       0.00      0.00      0.00         1
        8251       0.00      0.00      0.00         1
        8252       0.00      0.00      0.00         1
        8268       0.00      0.00      0.00         1
        8272       0.00      0.00      0.00         1
        8273       0.00      0.00      0.00         1
        8279       0.00      0.00      0.00         1
        8281       0.00      0.00      0.00         1
        8285       0.00      0.00      0.00         1
        8286       0.00      0.00      0.00         4
        8291       0.00      0.00      0.00         2
        8304       0.00      0.00      0.00         1
        8310       0.00      0.00      0.00         1
        8317       0.00      0.00      0.00         1
        8318       0.00      0.00      0.00         1
        8345       0.00      0.00      0.00         1
        8350       0.00      0.00      0.00         2
        8359       0.00      0.00      0.00         2
        8361       0.00      0.00      0.00         1
        8367       0.00      0.00      0.00         2
        8372       0.00      0.00      0.00         1
        8376       0.00      0.00      0.00         1
        8378       0.00      0.00      0.00         1
        8380       0.00      0.00      0.00         3
        8384       0.00      0.00      0.00         1
        8398       0.00      0.00      0.00         1
        8402       0.00      0.00      0.00         1
        8459       0.00      0.00      0.00         1
        8483       0.00      0.00      0.00         1
        8527       0.00      0.00      0.00         1
        8534       0.00      0.00      0.00         1
        8536       0.00      0.00      0.00         3
        8561       0.00      0.00      0.00         1
        8562       0.00      0.00      0.00         1
        8565       0.00      0.00      0.00         2
        8597       0.00      0.00      0.00         2
        8612       0.00      0.00      0.00         1
        8613       0.00      0.00      0.00         3
        8617       0.00      0.00      0.00         2
        8624       0.00      0.00      0.00         1
        8628       0.00      0.00      0.00         1
        8637       0.00      0.00      0.00         1
        8638       0.00      0.00      0.00         1
        8639       0.00      0.00      0.00         1
        8647       0.00      0.00      0.00         2
        8662       0.00      0.00      0.00         1
        8687       0.00      0.00      0.00         1
        8688       0.00      0.00      0.00         1
        8705       0.00      0.00      0.00         2
        8736       0.00      0.00      0.00         2
        8755       0.00      0.00      0.00         1
        8762       0.00      0.00      0.00         2
        8763       0.00      0.00      0.00         1
        8764       0.00      0.00      0.00         1
        8766       0.00      0.00      0.00         2
        8790       0.00      0.00      0.00         1
        8808       0.00      0.00      0.00         1
        8812       0.00      0.00      0.00         4
        8815       0.00      0.00      0.00         1
        8817       0.00      0.00      0.00         2
        8826       0.00      0.00      0.00         2
        8830       0.00      0.00      0.00         1
        8832       0.00      0.00      0.00         1
        8833       0.00      0.00      0.00         1
        8834       0.00      0.00      0.00         1
        8835       0.00      0.00      0.00         1
        8836       0.00      0.00      0.00         1
        8860       0.00      0.00      0.00         1
        8862       0.00      0.00      0.00         1
        8863       0.00      0.00      0.00         2
        8876       0.00      0.00      0.00         1
        8884       0.00      0.00      0.00         1
        8885       0.00      0.00      0.00         1
        8890       0.00      0.00      0.00         2
        8891       0.00      0.00      0.00         2
        8895       0.00      0.00      0.00         1
        8901       0.00      0.00      0.00         3
        8903       0.00      0.00      0.00         3
        8904       0.00      0.00      0.00         1
        8907       0.00      0.00      0.00         1
        8919       0.00      0.00      0.00         1
        8920       0.00      0.00      0.00         1
        8926       0.00      0.00      0.00         1
        8928       0.00      0.00      0.00         2
        8952       0.00      0.00      0.00         1
        8968       0.00      0.00      0.00         1
        8972       0.00      0.00      0.00         1
        8979       0.00      0.00      0.00         1
        8987       0.00      0.00      0.00         1
        8993       0.00      0.00      0.00         1
        9013       0.00      0.00      0.00         1
        9047       0.00      0.00      0.00         1
        9055       0.00      0.00      0.00         2
        9060       0.00      0.00      0.00         1
        9062       0.00      0.00      0.00         2
        9068       0.00      0.00      0.00         3
        9069       0.00      0.00      0.00         2
        9077       0.00      0.00      0.00         1
        9081       0.00      0.00      0.00         1
        9105       0.00      0.00      0.00         1
        9111       0.00      0.00      0.00         4
        9128       0.00      0.00      0.00         1
        9129       0.00      0.00      0.00         1
        9153       0.00      0.00      0.00         3
        9155       0.00      0.00      0.00         1
        9170       0.00      0.00      0.00         1
        9177       0.00      0.00      0.00         1
        9179       0.00      0.00      0.00         1
        9192       0.00      0.00      0.00         1
        9196       0.00      0.00      0.00         2
        9207       0.00      0.00      0.00         3
        9210       0.00      0.00      0.00         1
        9213       0.00      0.00      0.00         2
        9214       0.00      0.00      0.00         1
        9215       0.00      0.00      0.00         1
        9227       0.00      0.00      0.00         1
        9238       0.00      0.00      0.00         1
        9255       0.00      0.00      0.00         1
        9263       0.00      0.00      0.00         2
        9276       0.00      0.00      0.00         1
        9277       0.00      0.00      0.00         1
        9283       0.00      0.00      0.00         1
        9285       0.00      0.00      0.00         1
        9292       0.00      0.00      0.00         1
        9307       0.00      0.00      0.00         1
        9308       0.00      0.00      0.00         1
        9311       0.00      0.00      0.00         1
        9334       0.00      0.00      0.00         1
        9356       0.00      0.00      0.00         1
        9374       0.00      0.00      0.00         1
        9376       0.00      0.00      0.00         1
        9389       0.00      0.00      0.00         1
        9391       0.00      0.00      0.00         1
        9415       0.00      0.00      0.00         2
        9416       0.00      0.00      0.00         1
        9419       0.00      0.00      0.00         1
        9427       0.00      0.00      0.00         1
        9434       0.00      0.00      0.00         2
        9444       0.00      0.00      0.00         1
        9471       0.00      0.00      0.00         1
        9473       0.00      0.00      0.00         1
        9505       0.00      0.00      0.00         1
        9511       0.00      0.00      0.00         1
        9515       0.00      0.00      0.00         1
        9516       0.00      0.00      0.00         1
        9520       0.00      0.00      0.00         1
        9539       0.00      0.00      0.00         1
        9540       0.00      0.00      0.00         1
        9544       0.00      0.00      0.00         1
        9549       0.00      0.00      0.00         6
        9550       0.00      0.00      0.00         3
        9555       0.00      0.00      0.00         1
        9615       0.00      0.00      0.00         1
        9636       0.00      0.00      0.00         1
        9640       0.00      0.00      0.00         1
        9644       0.00      0.00      0.00         1
        9656       0.00      0.00      0.00         1
        9672       0.00      0.00      0.00         1
        9675       0.00      0.00      0.00         1
        9681       0.00      0.00      0.00         3
        9682       0.00      0.00      0.00         1
        9683       0.00      0.00      0.00         1
        9716       0.00      0.00      0.00         1
        9717       0.00      0.00      0.00         1
        9725       0.00      0.00      0.00         1
        9729       0.00      0.00      0.00         1
        9758       0.00      0.00      0.00         1
        9759       0.00      0.00      0.00         1
        9775       0.00      0.00      0.00         1
        9806       0.00      0.00      0.00         2
        9877       0.00      0.00      0.00         1
        9893       0.00      0.00      0.00         1
        9955       0.00      0.00      0.00         3
        9963       0.00      0.00      0.00         2
        9970       0.00      0.00      0.00         2
       10000       0.00      0.00      0.00         1
       10002       0.00      0.00      0.00         3
       10004       0.00      0.00      0.00         2
       10009       0.00      0.00      0.00         1
       10019       0.00      0.00      0.00         2
       10028       0.00      0.00      0.00         1
       10031       0.00      0.00      0.00         1
       10032       0.00      0.00      0.00         2
       10062       0.00      0.00      0.00         3
       10068       0.00      0.00      0.00         1
       10070       0.00      0.00      0.00         1
       10073       0.00      0.00      0.00         1
       10083       0.00      0.00      0.00         1
       10105       0.00      0.00      0.00         1
       10106       0.00      0.00      0.00         1
       10107       0.00      0.00      0.00         1
       10157       0.00      0.00      0.00         1
       10159       0.00      0.00      0.00         1
       10163       0.00      0.00      0.00         1
       10164       0.00      0.00      0.00         1
       10165       0.00      0.00      0.00         1
       10182       0.00      0.00      0.00         1
       10191       0.00      0.00      0.00         1
       10200       0.00      0.00      0.00         2
       10212       0.00      0.00      0.00         2
       10213       0.00      0.00      0.00         1
       10220       0.00      0.00      0.00         1
       10221       0.00      0.00      0.00         1
       10225       0.00      0.00      0.00         1
       10230       0.00      0.00      0.00         3
       10236       0.00      0.00      0.00         1
       10244       0.00      0.00      0.00         3
       10256       0.00      0.00      0.00         1
       10261       0.00      0.00      0.00         2
       10267       0.00      0.00      0.00         4
       10278       0.00      0.00      0.00         1
       10280       0.00      0.00      0.00         1
       10305       0.00      0.00      0.00         1
       10331       0.00      0.00      0.00         1
       10336       0.00      0.00      0.00         1
       10340       0.00      0.00      0.00         1
       10341       0.00      0.00      0.00         1
       10342       0.00      0.00      0.00         2
       10346       0.00      0.00      0.00         1
       10427       0.00      0.00      0.00         1
       10451       0.00      0.00      0.00         1
       10477       0.00      0.00      0.00         1
       10481       0.00      0.00      0.00         1
       10540       0.00      0.00      0.00         1
       10555       0.00      0.00      0.00         1
       10559       0.00      0.00      0.00         1
       10572       0.00      0.00      0.00         1
       10576       0.00      0.00      0.00         1
       10577       0.00      0.00      0.00         1
       10585       0.00      0.00      0.00         1
       10600       0.00      0.00      0.00         1
       10656       0.00      0.00      0.00         1
       10658       0.00      0.00      0.00         1
       10674       0.00      0.00      0.00         1
       10712       0.00      0.00      0.00         1
       10750       0.00      0.00      0.00         1
       10752       0.00      0.00      0.00         1
       10765       0.00      0.00      0.00         2
       10776       0.00      0.00      0.00         1
       10786       0.00      0.00      0.00         1
       10792       0.00      0.00      0.00         1
       10825       0.00      0.00      0.00         1
       10828       0.00      0.00      0.00         1
       10849       0.00      0.00      0.00         1
       10859       0.00      0.00      0.00         1
       10860       0.00      0.00      0.00         1
       10871       0.00      0.00      0.00         1
       10876       0.00      0.00      0.00         1
       10879       0.00      0.00      0.00         1
       10902       0.00      0.00      0.00         1
       10904       0.00      0.00      0.00         1
       10940       0.00      0.00      0.00         2
       10956       0.00      0.00      0.00         3
       10966       0.00      0.00      0.00         1
       10974       0.00      0.00      0.00         1
       10979       0.00      0.00      0.00         2
       10988       0.00      0.00      0.00         1
       10996       0.00      0.00      0.00         1
       11008       0.00      0.00      0.00         1
       11027       0.00      0.00      0.00         1
       11060       0.00      0.00      0.00         2
       11071       0.00      0.00      0.00         1
       11093       0.00      0.00      0.00         1
       11108       0.00      0.00      0.00         1
       11155       0.00      0.00      0.00         1
       11159       0.00      0.00      0.00         2
       11191       0.00      0.00      0.00         1
       11198       0.00      0.00      0.00         1
       11216       0.00      0.00      0.00         1
       11257       0.00      0.00      0.00         3
       11263       0.00      0.00      0.00         1
       11278       0.00      0.00      0.00         1
       11282       0.00      0.00      0.00         2
       11338       0.00      0.00      0.00         1
       11347       0.00      0.00      0.00         1
       11349       0.00      0.00      0.00         1
       11351       0.00      0.00      0.00         1
       11366       0.00      0.00      0.00         1
       11435       0.00      0.00      0.00         1
       11464       0.00      0.00      0.00         1
       11467       0.00      0.00      0.00         1
       11492       0.00      0.00      0.00         1
       11494       0.00      0.00      0.00         2
       11525       0.00      0.00      0.00         1
       11538       0.00      0.00      0.00         1
       11547       0.00      0.00      0.00         1
       11548       0.00      0.00      0.00         1
       11549       0.00      0.00      0.00         1
       11563       0.00      0.00      0.00         1
       11591       0.00      0.00      0.00         1
       11595       0.00      0.00      0.00         2
       11602       0.00      0.00      0.00         2
       11626       0.00      0.00      0.00         1
       11639       0.00      0.00      0.00         2
       11644       0.00      0.00      0.00         1
       11650       0.00      0.00      0.00         1
       11681       0.00      0.00      0.00         1
       11693       0.00      0.00      0.00         2
       11709       0.00      0.00      0.00         1
       11717       0.00      0.00      0.00         1
       11734       0.00      0.00      0.00         1
       11792       0.00      0.00      0.00         1
       11854       0.00      0.00      0.00         1
       11887       0.00      0.00      0.00         1
       11904       0.00      0.00      0.00         2
       11911       0.00      0.00      0.00         2
       11937       0.00      0.00      0.00         1
       11984       0.00      0.00      0.00         1
       11986       0.00      0.00      0.00         1
       12000       0.00      0.00      0.00         1
       12035       0.00      0.00      0.00         2
       12047       0.00      0.00      0.00         2
       12059       0.00      0.00      0.00         1
       12063       0.00      0.00      0.00         1
       12073       0.00      0.00      0.00         1
       12087       0.00      0.00      0.00         1
       12089       0.00      0.00      0.00         1
       12101       0.00      0.00      0.00         2
       12113       0.00      0.00      0.00         2
       12135       0.00      0.00      0.00         1
       12144       0.00      0.00      0.00         1
       12158       0.00      0.00      0.00         1
       12186       0.00      0.00      0.00         1
       12210       0.00      0.00      0.00         1
       12232       0.00      0.00      0.00         1
       12235       0.00      0.00      0.00         1
       12261       0.00      0.00      0.00         1
       12327       0.00      0.00      0.00         1
       12343       0.00      0.00      0.00         2
       12409       0.00      0.00      0.00         1
       12432       0.00      0.00      0.00         1
       12472       0.00      0.00      0.00         1
       12475       0.00      0.00      0.00         1
       12476       0.00      0.00      0.00         4
       12477       0.00      0.00      0.00         1
       12478       0.00      0.00      0.00         1
       12479       0.00      0.00      0.00         1
       12487       0.00      0.00      0.00         1
       12497       0.00      0.00      0.00         1
       12525       0.00      0.00      0.00         1
       12530       0.00      0.00      0.00         2
       12532       0.00      0.00      0.00         1
       12551       0.00      0.00      0.00         1
       12578       0.00      0.00      0.00         1
       12611       0.00      0.00      0.00         1
       12620       0.00      0.00      0.00         1
       12627       0.00      0.00      0.00         1
       12678       0.00      0.00      0.00         1
       12729       0.00      0.00      0.00         1
       12738       0.00      0.00      0.00         1
       12739       0.00      0.00      0.00         3
       12759       0.00      0.00      0.00         3
       12761       0.00      0.00      0.00         1
       12809       0.00      0.00      0.00         2
       12832       0.00      0.00      0.00         1
       12833       0.00      0.00      0.00         1
       12962       0.00      0.00      0.00         1
       13029       0.00      0.00      0.00         1
       13047       0.00      0.00      0.00         1
       13048       0.00      0.00      0.00         1
       13087       0.00      0.00      0.00         1
       13099       0.00      0.00      0.00         3
       13113       0.00      0.00      0.00         1
       13166       0.00      0.00      0.00         1
       13231       0.00      0.00      0.00         1
       13269       0.00      0.00      0.00         1
       13299       0.00      0.00      0.00         1
       13303       0.00      0.00      0.00         2
       13326       0.00      0.00      0.00         1
       13350       0.00      0.00      0.00         1
       13353       0.00      0.00      0.00         2
       13361       0.00      0.00      0.00         1
       13373       0.00      0.00      0.00         1
       13379       0.00      0.00      0.00         4
       13396       0.00      0.00      0.00         2
       13417       0.00      0.00      0.00         1
       13419       0.00      0.00      0.00         2
       13432       0.00      0.00      0.00         1
       13433       0.00      0.00      0.00         1
       13436       0.00      0.00      0.00         1
       13456       0.00      0.00      0.00         2
       13464       0.00      0.00      0.00         1
       13481       0.00      0.00      0.00         1
       13561       0.00      0.00      0.00         1
       13606       0.00      0.00      0.00         1
       13660       0.00      0.00      0.00         2
       13668       0.00      0.00      0.00         1
       13685       0.00      0.00      0.00         1
       13688       0.00      0.00      0.00         1
       13700       0.00      0.00      0.00         1
       13717       0.00      0.00      0.00         1
       13724       0.00      0.00      0.00         2
       13745       0.00      0.00      0.00         1
       13747       0.00      0.00      0.00         1
       13750       0.00      0.00      0.00         1
       13826       0.00      0.00      0.00         1
       13831       0.00      0.00      0.00         1
       13876       0.00      0.00      0.00         4
       13878       0.00      0.00      0.00         1
       13880       0.00      0.00      0.00         1
       13889       0.00      0.00      0.00         1
       13900       0.00      0.00      0.00         1
       13937       0.00      0.00      0.00         1
       13939       0.00      0.00      0.00         1
       13958       0.00      0.00      0.00         1
       13985       0.00      0.00      0.00         1
       13995       0.00      0.00      0.00         1
       14009       0.00      0.00      0.00         1
       14038       0.00      0.00      0.00         1
       14053       0.00      0.00      0.00         1
       14074       0.00      0.00      0.00         1
       14079       0.00      0.00      0.00         1
       14096       0.00      0.00      0.00         1
       14110       0.00      0.00      0.00         1
       14122       0.00      0.00      0.00         1
       14178       0.00      0.00      0.00         1
       14188       0.00      0.00      0.00         1
       14192       0.00      0.00      0.00         1
       14279       0.00      0.00      0.00         1
       14281       0.00      0.00      0.00         1
       14286       0.00      0.00      0.00         1
       14309       0.00      0.00      0.00         1
       14320       0.00      0.00      0.00         1
       14404       0.00      0.00      0.00         1
       14406       0.00      0.00      0.00         1
       14409       0.00      0.00      0.00         1
       14448       0.00      0.00      0.00         1
       14523       0.00      0.00      0.00         1
       14606       0.00      0.00      0.00         1
       14616       0.00      0.00      0.00         1
       14657       0.00      0.00      0.00         1
       14696       0.00      0.00      0.00         2
       14730       0.00      0.00      0.00         3
       14751       0.00      0.00      0.00         1
       14787       0.00      0.00      0.00         2
       14789       0.00      0.00      0.00         2
       14912       0.00      0.00      0.00         1
       15196       0.00      0.00      0.00         1
       15224       0.00      0.00      0.00         2
       15246       0.00      0.00      0.00         1
       15249       0.00      0.00      0.00         4
       15252       0.00      0.00      0.00         1
       15267       0.00      0.00      0.00         2
       15268       0.00      0.00      0.00         1
       15276       0.00      0.00      0.00         2
       15289       0.00      0.00      0.00         1
       15306       0.00      0.00      0.00         3
       15328       0.00      0.00      0.00         2
       15335       0.00      0.00      0.00         1
       15373       0.00      0.00      0.00         2
       15378       0.00      0.00      0.00         2
       15415       0.00      0.00      0.00         1
       15428       0.00      0.00      0.00         1
       15495       0.00      0.00      0.00         2
       15512       0.00      0.00      0.00         1
       15601       0.00      0.00      0.00         1
       15616       0.00      0.00      0.00         1
       15657       0.00      0.00      0.00         1
       15695       0.00      0.00      0.00         1
       15698       0.00      0.00      0.00         1
       15707       0.00      0.00      0.00         2
       15721       0.00      0.00      0.00         1
       15792       0.00      0.00      0.00         1
       15801       0.00      0.00      0.00         1
       15821       0.00      0.00      0.00         2
       15838       0.00      0.00      0.00         1
       15861       0.00      0.00      0.00         1
       15867       0.00      0.00      0.00         3
       15934       0.00      0.00      0.00         1
       15972       0.00      0.00      0.00         1
       15996       0.00      0.00      0.00         1
       16042       0.00      0.00      0.00         1
       16053       0.00      0.00      0.00         1
       16123       0.00      0.00      0.00         1
       16160       0.00      0.00      0.00         1
       16185       0.00      0.00      0.00         1
       16194       0.00      0.00      0.00         1
       16198       0.00      0.00      0.00         2
       16235       0.00      0.00      0.00         2
       16242       0.00      0.00      0.00         1
       16245       0.00      0.00      0.00         1
       16246       0.00      0.00      0.00         2
       16257       0.00      0.00      0.00         1
       16322       0.00      0.00      0.00         2
       16331       0.00      0.00      0.00         1
       16345       0.00      0.00      0.00         1
       16372       0.00      0.00      0.00         1
       16409       0.00      0.00      0.00         1
       16422       0.00      0.00      0.00         1
       16457       0.00      0.00      0.00         2
       16476       0.00      0.00      0.00         1
       16502       0.00      0.00      0.00         1
       16505       0.00      0.00      0.00         1
       16544       0.00      0.00      0.00         2
       16550       0.00      0.00      0.00         1
       16602       0.00      0.00      0.00         7
       16614       0.00      0.00      0.00         1
       16620       0.00      0.00      0.00         1
       16654       0.00      0.00      0.00         1
       16662       0.00      0.00      0.00         1
       16691       0.00      0.00      0.00         1
       16708       0.00      0.00      0.00         1
       16710       0.00      0.00      0.00         1
       16726       0.00      0.00      0.00         1
       16728       0.00      0.00      0.00         1
       16740       0.00      0.00      0.00         1
       16785       0.00      0.00      0.00         1
       16786       0.00      0.00      0.00         2
       16800       0.00      0.00      0.00         1
       16812       0.00      0.00      0.00         1
       16846       0.00      0.00      0.00         1
       16865       0.00      0.00      0.00         2
       16903       0.00      0.00      0.00         1
       16906       0.00      0.00      0.00         1
       16933       0.00      0.00      0.00         1
       16943       0.00      0.00      0.00         2
       16960       0.00      0.00      0.00         1
       17056       0.00      0.00      0.00         1
       17117       0.00      0.00      0.00         1
       17153       0.00      0.00      0.00         1
       17191       0.00      0.00      0.00         2
       17222       0.00      0.00      0.00         1
       17229       0.00      0.00      0.00         1
       17256       0.00      0.00      0.00         1
       17284       0.00      0.00      0.00         1
       17288       0.00      0.00      0.00         1
       17332       0.00      0.00      0.00         1
       17337       0.00      0.00      0.00         1
       17415       0.00      0.00      0.00         1
       17437       0.00      0.00      0.00         1
       17440       0.00      0.00      0.00         2
       17446       0.00      0.00      0.00         1
       17460       0.00      0.00      0.00         1
       17487       0.00      0.00      0.00         1
       17513       0.00      0.00      0.00         1
       17533       0.00      0.00      0.00         2
       17605       0.00      0.00      0.00         1
       17608       0.00      0.00      0.00         1
       17617       0.00      0.00      0.00         1
       17667       0.00      0.00      0.00         1
       17680       0.00      0.00      0.00         1
       17715       0.00      0.00      0.00         1
       17753       0.00      0.00      0.00         1
       17766       0.00      0.00      0.00         1
       17792       0.00      0.00      0.00         1
       17807       0.00      0.00      0.00         1
       17809       0.00      0.00      0.00         1
       17814       0.00      0.00      0.00         1
       17830       0.00      0.00      0.00         1
       17843       0.00      0.00      0.00         2
       17895       0.00      0.00      0.00         1
       17902       0.00      0.00      0.00         7
       17904       0.00      0.00      0.00         1
       17972       0.00      0.00      0.00         1
       18002       0.00      0.00      0.00         1
       18031       0.00      0.00      0.00         1
       18074       0.00      0.00      0.00         1
       18140       0.00      0.00      0.00         1
       18150       0.00      0.00      0.00         1
       18197       0.00      0.00      0.00         1
       18198       0.00      0.00      0.00         1
       18239       0.00      0.00      0.00         2
       18276       0.00      0.00      0.00         1
       18307       0.00      0.00      0.00         1
       18344       0.00      0.00      0.00         2
       18485       0.00      0.00      0.00         1
       18519       0.00      0.00      0.00         1
       18522       0.00      0.00      0.00         1
       18544       0.00      0.00      0.00         1
       18622       0.00      0.00      0.00         1
       18634       0.00      0.00      0.00         1
       18671       0.00      0.00      0.00         1
       18680       0.00      0.00      0.00         1
       18687       0.00      0.00      0.00         2
       18699       0.00      0.00      0.00         1
       18714       0.00      0.00      0.00         1
       18724       0.00      0.00      0.00         1
       18725       0.00      0.00      0.00         3
       18727       0.00      0.00      0.00         2
       18728       0.00      0.00      0.00         3
       18729       0.00      0.00      0.00         1
       18732       0.00      0.00      0.00         1
       18739       0.00      0.00      0.00         1
       18744       0.00      0.00      0.00         1
       18752       0.00      0.00      0.00         2
       18759       0.00      0.00      0.00         1
       18762       0.00      0.00      0.00         6
       18763       0.00      0.00      0.00         3
       18764       0.00      0.00      0.00         1
       18766       0.00      0.00      0.00         1
       18781       0.00      0.00      0.00         1
       18785       0.00      0.00      0.00         1
       18825       0.00      0.00      0.00         1
       18849       0.00      0.00      0.00         1
       18912       0.00      0.00      0.00         1
       18928       0.00      0.00      0.00         1
       18958       0.00      0.00      0.00         1
       19049       0.00      0.00      0.00         5
       19051       0.00      0.00      0.00         2
       19057       0.00      0.00      0.00         1
       19064       0.00      0.00      0.00         2
       19070       0.00      0.00      0.00         2
       19075       0.00      0.00      0.00         2
       19093       0.00      0.00      0.00         3
       19095       0.00      0.00      0.00         1
       19117       0.00      0.00      0.00         1
       19120       0.00      0.00      0.00         1
       19142       0.00      0.00      0.00         1
       19261       0.00      0.00      0.00         1
       19263       0.00      0.00      0.00         1
       19301       0.00      0.00      0.00         3
       19352       0.00      0.00      0.00         1
       19359       0.00      0.00      0.00         2
       19380       0.00      0.00      0.00         1
       19385       0.00      0.00      0.00         1
       19408       0.00      0.00      0.00         1
       19423       0.00      0.00      0.00         8
       19430       0.00      0.00      0.00         3
       19439       0.00      0.00      0.00         3
       19440       0.00      0.00      0.00         1
       19442       0.00      0.00      0.00         2
       19445       0.00      0.00      0.00         1
       19446       0.00      0.00      0.00         1
       19448       0.00      0.00      0.00         1
       19478       0.00      0.00      0.00         3
       19480       0.00      0.00      0.00         1
       19481       0.00      0.00      0.00         1
       19484       0.00      0.00      0.00         2
       19496       0.00      0.00      0.00         1
       19501       0.00      0.00      0.00         1
       19522       0.00      0.00      0.00         1
       19525       0.00      0.00      0.00         1
       19526       0.00      0.00      0.00         2
       19542       0.00      0.00      0.00         1
       19565       0.00      0.00      0.00         1
       19607       0.00      0.00      0.00         1
       19636       0.00      0.00      0.00         1
       19690       0.00      0.00      0.00         1
       19699       0.00      0.00      0.00         1
       19715       0.00      0.00      0.00         1
       19797       0.00      0.00      0.00         1
       19809       0.00      0.00      0.00         1
       19817       0.00      0.00      0.00         1
       19818       0.00      0.00      0.00         1
       19824       0.00      0.00      0.00         1
       19856       0.00      0.00      0.00         1
       19895       0.00      0.00      0.00         1
       19916       0.00      0.00      0.00         2
       19920       0.00      0.00      0.00         1
       19944       0.00      0.00      0.00         1
       19946       0.00      0.00      0.00         1
       19951       0.00      0.00      0.00         1
       19952       0.00      0.00      0.00         1
       19967       0.00      0.00      0.00         1
       19986       0.00      0.00      0.00         1
       20061       0.00      0.00      0.00         1
       20093       0.00      0.00      0.00         1
       20232       0.00      0.00      0.00         1
       20239       0.00      0.00      0.00         2
       20254       0.00      0.00      0.00         1
       20274       0.00      0.00      0.00         1
       20329       0.00      0.00      0.00         1
       20338       0.00      0.00      0.00         1
       20377       0.00      0.00      0.00         1
       20404       0.00      0.00      0.00         2
       20512       0.00      0.00      0.00         1
       20548       0.00      0.00      0.00         1
       20606       0.00      0.00      0.00         1
       20659       0.00      0.00      0.00         5
       20753       0.00      0.00      0.00         1
       20757       0.00      0.00      0.00         1
       20765       0.00      0.00      0.00         1
       20770       0.00      0.00      0.00         1
       20771       0.00      0.00      0.00         1
       20834       0.00      0.00      0.00         2
       20902       0.00      0.00      0.00         1
       20958       0.00      0.00      0.00         1
       21027       0.00      0.00      0.00         1
       21138       0.00      0.00      0.00         1
       21159       0.00      0.00      0.00         1
       21290       0.00      0.00      0.00         1
       21445       0.00      0.00      0.00         1
       21457       0.00      0.00      0.00         1
       21768       0.00      0.00      0.00         1
       21771       0.00      0.00      0.00         2
       21796       0.00      0.00      0.00         1
       21829       0.00      0.00      0.00         1
       21946       0.00      0.00      0.00         1
       22086       0.00      0.00      0.00         1
       22152       0.00      0.00      0.00         1
       22299       0.00      0.00      0.00         1
       22305       0.00      0.00      0.00         1
       22361       0.00      0.00      0.00         1
       22372       0.00      0.00      0.00         1
       22375       0.00      0.00      0.00         1
       22392       0.00      0.00      0.00         1
       22480       0.00      0.00      0.00         1
       22507       0.00      0.00      0.00         3
       22553       0.00      0.00      0.00         1
       22594       0.00      0.00      0.00         1
       22612       0.00      0.00      0.00         1
       22616       0.00      0.00      0.00         2
       22663       0.00      0.00      0.00         2
       22676       0.00      0.00      0.00         1
       22680       0.00      0.00      0.00         1
       22703       0.00      0.00      0.00         1
       22726       0.00      0.00      0.00         1
       22796       0.00      0.00      0.00         1
       22847       0.00      0.00      0.00         1
       22858       0.00      0.00      0.00         1
       22938       0.00      0.00      0.00         2
       22997       0.00      0.00      0.00         1
       23121       0.00      0.00      0.00         2
       23178       0.00      0.00      0.00         1
       23228       0.00      0.00      0.00         2
       23245       0.00      0.00      0.00         1
       23271       0.00      0.00      0.00         1
       23376       0.00      0.00      0.00         2
       23406       0.00      0.00      0.00         2
       23432       0.00      0.00      0.00         1
       23506       0.00      0.00      0.00         1
       23517       0.00      0.00      0.00         1
       23641       0.00      0.00      0.00         1
       23952       0.00      0.00      0.00         1
       23959       0.00      0.00      0.00         1
       23984       0.00      0.00      0.00         1
       24086       0.00      0.00      0.00         1
       24104       0.00      0.00      0.00         1
       24218       0.00      0.00      0.00         1
       24235       0.00      0.00      0.00         1
       24293       0.00      0.00      0.00         1
       24342       0.00      0.00      0.00         1
       24350       0.00      0.00      0.00         1
       24352       0.00      0.00      0.00         1
       24458       0.00      0.00      0.00         1
       24465       0.00      0.00      0.00         1
       24603       0.00      0.00      0.00         1
       24711       0.00      0.00      0.00         1
       24729       0.00      0.00      0.00         2
       24755       0.00      0.00      0.00         1
       24951       0.00      0.00      0.00         2
       24952       0.00      0.00      0.00         4
       24961       0.00      0.00      0.00         3
       25003       0.00      0.00      0.00         1
       25047       0.00      0.00      0.00         4
       25109       0.00      0.00      0.00         1
       25125       0.00      0.00      0.00         2
       25199       0.00      0.00      0.00         1
       25222       0.00      0.00      0.00         2
       25227       0.00      0.00      0.00         1
       25235       0.00      0.00      0.00         1
       25253       0.00      0.00      0.00         1
       25254       0.00      0.00      0.00         1
       25264       0.00      0.00      0.00         1
       25281       0.00      0.00      0.00         5
       25342       0.00      0.00      0.00         1
       25409       0.00      0.00      0.00         2
       25417       0.00      0.00      0.00         1
       25442       0.00      0.00      0.00         1
       25446       0.00      0.00      0.00         2
       25472       0.00      0.00      0.00         1
       25532       0.00      0.00      0.00         1
       25550       0.00      0.00      0.00         1
       25715       0.00      0.00      0.00         2
       25741       0.00      0.00      0.00         2
       25906       0.00      0.00      0.00         1
       25944       0.00      0.00      0.00         1
       25959       0.00      0.00      0.00         1
       26013       0.00      0.00      0.00         2
       26038       0.00      0.00      0.00         1
       26054       0.00      0.00      0.00         1
       26541       0.00      0.00      0.00         1
       26543       0.00      0.00      0.00         1
       26747       0.00      0.00      0.00         1
       26784       0.00      0.00      0.00         1
       26829       0.00      0.00      0.00         1
       26843       0.00      0.00      0.00         1
       26850       0.00      0.00      0.00         1
       26885       0.00      0.00      0.00         1
       26913       0.00      0.00      0.00         1
       26949       0.00      0.00      0.00         1
       27210       0.00      0.00      0.00         1
       27458       0.00      0.00      0.00         1
       27640       0.00      0.00      0.00         1
       27646       0.00      0.00      0.00         1
       27653       0.00      0.00      0.00         1
       27670       0.00      0.00      0.00         1
       27707       0.00      0.00      0.00         1
       27781       0.00      0.00      0.00         1
       27797       0.00      0.00      0.00         1
       27826       0.00      0.00      0.00         1
       27874       0.00      0.00      0.00         2
       27961       0.00      0.00      0.00         1
       28108       0.00      0.00      0.00         1
       28238       0.00      0.00      0.00         1
       28293       0.00      0.00      0.00         1
       28409       0.00      0.00      0.00         1
       28420       0.00      0.00      0.00         1
       28464       0.00      0.00      0.00         1
       28465       0.00      0.00      0.00         1
       28468       0.00      0.00      0.00         1
       28504       0.00      0.00      0.00         1
       28547       0.00      0.00      0.00         2
       28612       0.00      0.00      0.00         1
       28649       0.00      0.00      0.00         1
       28696       0.00      0.00      0.00         1
       28699       0.00      0.00      0.00         1
       28835       0.00      0.00      0.00         1
       28844       0.00      0.00      0.00         1
       28848       0.00      0.00      0.00         1
       29066       0.00      0.00      0.00         2
       29271       0.00      0.00      0.00         1
       29287       0.00      0.00      0.00         1
       29335       0.00      0.00      0.00         1
       29361       0.00      0.00      0.00         1
       29441       0.00      0.00      0.00         2
       29463       0.00      0.00      0.00         1
       29715       0.00      0.00      0.00         1
       29742       0.00      0.00      0.00         2
       29839       0.00      0.00      0.00         2
       29928       0.00      0.00      0.00         1
       30022       0.00      0.00      0.00         1
       30120       0.00      0.00      0.00         1
       30136       0.00      0.00      0.00         1
       30282       0.00      0.00      0.00         1
       30286       0.00      0.00      0.00         1
       30349       0.00      0.00      0.00         1
       30367       0.00      0.00      0.00         1
       30373       0.00      0.00      0.00         1
       30394       0.00      0.00      0.00         1
       30485       0.00      0.00      0.00         1
       30629       0.00      0.00      0.00         1
       30652       0.00      0.00      0.00         1
       30791       0.00      0.00      0.00         1
       30828       0.00      0.00      0.00         1
       30882       0.00      0.00      0.00         1
       31020       0.00      0.00      0.00         1
       31108       0.00      0.00      0.00         2
       31129       0.00      0.00      0.00         2
       31472       0.00      0.00      0.00         1
       31482       0.00      0.00      0.00         1
       31675       0.00      0.00      0.00         2
       31805       0.00      0.00      0.00         1
       31817       0.00      0.00      0.00         1
       31903       0.00      0.00      0.00         1
       31947       0.00      0.00      0.00         1
       32105       0.00      0.00      0.00         1
       32176       0.00      0.00      0.00         1
       32185       0.00      0.00      0.00         1
       32247       0.00      0.00      0.00         4
       32296       0.00      0.00      0.00         2
       32421       0.00      0.00      0.00         1
       32427       0.00      0.00      0.00         1
       32437       0.00      0.00      0.00         1
       32783       0.00      0.00      0.00         1
       32802       0.00      0.00      0.00         1
       32822       0.00      0.00      0.00         1
       32854       0.00      0.00      0.00         1
       32909       0.00      0.00      0.00         1
       33067       0.00      0.00      0.00         2
       33084       0.00      0.00      0.00         1
       33116       0.00      0.00      0.00         2
       33147       0.00      0.00      0.00         1
       33282       0.00      0.00      0.00         1
       33386       0.00      0.00      0.00         1
       33496       0.00      0.00      0.00         1
       33542       0.00      0.00      0.00         1
       33631       0.00      0.00      0.00         1
       33688       0.00      0.00      0.00         1
       33907       0.00      0.00      0.00         1
       33925       0.00      0.00      0.00         1
       33983       0.00      0.00      0.00         2
       34095       0.00      0.00      0.00         2
       34343       0.00      0.00      0.00         1
       35000       0.00      0.00      0.00         1
       35280       0.00      0.00      0.00         1
       35318       0.00      0.00      0.00         2
       35620       0.00      0.00      0.00         1
       35873       0.00      0.00      0.00         1
       36091       0.00      0.00      0.00         1
       36149       0.00      0.00      0.00         1
       36234       0.00      0.00      0.00         1
       36639       0.00      0.00      0.00         1
       36756       0.00      0.00      0.00         2
       36757       0.00      0.00      0.00         1
       36983       0.00      0.00      0.00         1
       37624       0.00      0.00      0.00         1
       37712       0.00      0.00      0.00         1
       37798       0.00      0.00      0.00         1
       37861       0.00      0.00      0.00         1
       37865       0.00      0.00      0.00         1
       37885       0.00      0.00      0.00         1
       37886       0.00      0.00      0.00         1
       37931       0.00      0.00      0.00         1
       38118       0.00      0.00      0.00         1
       38164       0.00      0.00      0.00         2
       38227       0.00      0.00      0.00         1
       38229       0.00      0.00      0.00         2
       38427       0.00      0.00      0.00         1
       38477       0.00      0.00      0.00         1
       38525       0.00      0.00      0.00         2
       38942       0.00      0.00      0.00         1
       39114       0.00      0.00      0.00         1
       39495       0.00      0.00      0.00         1
       39642       0.00      0.00      0.00         1
       39791       0.00      0.00      0.00         1
       40269       0.00      0.00      0.00         1
       40316       0.00      0.00      0.00         1
       40578       0.00      0.00      0.00         1
       41333       0.00      0.00      0.00         1
       41434       0.00      0.00      0.00         2
       41523       0.00      0.00      0.00         1
       41681       0.00      0.00      0.00         2
       41842       0.00      0.00      0.00         1
       42015       0.00      0.00      0.00         1
       42029       0.00      0.00      0.00         1
       42178       0.00      0.00      0.00         1
       42213       0.00      0.00      0.00         1
       42217       0.00      0.00      0.00         1
       42406       0.00      0.00      0.00         1
       42521       0.00      0.00      0.00         1
       43661       0.00      0.00      0.00         1
       43780       0.00      0.00      0.00         1
       43874       0.00      0.00      0.00         1
       44878       0.00      0.00      0.00         1
       45598       0.00      0.00      0.00         1
       45609       0.00      0.00      0.00         1
       46476       0.00      0.00      0.00         1
       46685       0.00      0.00      0.00         1
       47542       0.00      0.00      0.00         1
       47590       0.00      0.00      0.00         1
       47912       0.00      0.00      0.00         2
       47915       0.00      0.00      0.00         2
       48066       0.00      0.00      0.00         1
       48163       0.00      0.00      0.00         2
       48169       0.00      0.00      0.00         2
       48196       0.00      0.00      0.00         2
       48634       0.00      0.00      0.00         1
       48648       0.00      0.00      0.00         1
       48743       0.00      0.00      0.00         1
       48939       0.00      0.00      0.00         1
       49327       0.00      0.00      0.00         1
       49386       0.00      0.00      0.00         1
       49629       0.00      0.00      0.00         1
       50420       0.00      0.00      0.00         1
       50573       0.00      0.00      0.00         1
       50657       0.00      0.00      0.00         2
       50896       0.00      0.00      0.00         1
       50978       0.00      0.00      0.00         1
       51009       0.00      0.00      0.00         1
       51299       0.00      0.00      0.00         1
       51316       0.00      0.00      0.00         1
       52052       0.00      0.00      0.00         1
       52410       0.00      0.00      0.00         1
       52480       0.00      0.00      0.00         1
       52512       0.00      0.00      0.00         1
       52585       0.00      0.00      0.00         2
       52770       0.00      0.00      0.00         1
       52994       0.00      0.00      0.00         2
       53088       0.00      0.00      0.00         1
       54105       0.00      0.00      0.00         2
       54228       0.00      0.00      0.00         1
       54450       0.00      0.00      0.00         1
       54594       0.00      0.00      0.00         1
       54850       0.00      0.00      0.00         1
       55040       0.00      0.00      0.00         1
       55943       0.00      0.00      0.00         1
       56780       0.00      0.00      0.00         1
       57074       0.00      0.00      0.00         2
       57761       0.00      0.00      0.00         1
       59168       0.00      0.00      0.00         1
       59489       0.00      0.00      0.00         1
       59975       0.00      0.00      0.00         1
       60368       0.00      0.00      0.00         1
       60429       0.00      0.00      0.00         1
       60587       0.00      0.00      0.00         1
       61194       0.00      0.00      0.00         1
       61466       0.00      0.00      0.00         1
       61524       0.00      0.00      0.00         1
       61747       0.00      0.00      0.00         1
       62755       0.00      0.00      0.00         1
       63606       0.00      0.00      0.00         2
       65129       0.00      0.00      0.00         1
       65304       0.00      0.00      0.00         1
       65910       0.00      0.00      0.00         2
       66054       0.00      0.00      0.00         1
       67632       0.00      0.00      0.00         1
       67891       0.00      0.00      0.00         1
       68087       0.00      0.00      0.00         1
       68496       0.00      0.00      0.00         1
       68647       0.00      0.00      0.00         1
       69637       0.00      0.00      0.00         1
       69813       0.00      0.00      0.00         1
       70292       0.00      0.00      0.00         1
       70542       0.00      0.00      0.00         1
       70598       0.00      0.00      0.00         1
       70801       0.00      0.00      0.00         1
       70973       0.00      0.00      0.00         1
       71056       0.00      0.00      0.00         1
       71984       0.00      0.00      0.00         2
       72355       0.00      0.00      0.00         1
       73404       0.00      0.00      0.00         1
       73642       0.00      0.00      0.00         2
       73988       0.00      0.00      0.00         1
       78159       0.00      0.00      0.00         1

    accuracy                           0.01     10885
   macro avg       0.00      0.00      0.00     10885
weighted avg       0.00      0.01      0.00     10885


**Question:** How well does the logistic regression model predict the occurrence and severity of a storm, by a coded value location description across the US?

**Answer:** 

The model is attempting to predict the likeliness of a power outage by loation and date, however, the data has many limitations, and the model may 